In [1]:
import sys

In [2]:
sys.executable

'/Users/christanasescu/opt/anaconda3/bin/python3'

In [3]:
sys.path

['/Users/christanasescu',
 '/Users/christanasescu/opt/anaconda3/lib/python37.zip',
 '/Users/christanasescu/opt/anaconda3/lib/python3.7',
 '/Users/christanasescu/opt/anaconda3/lib/python3.7/lib-dynload',
 '',
 '/Users/christanasescu/opt/anaconda3/lib/python3.7/site-packages',
 '/Users/christanasescu/opt/anaconda3/lib/python3.7/site-packages/IPython/extensions',
 '/Users/christanasescu/.ipython']

In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
from nltk.tokenize import sent_tokenize, word_tokenize
import os
#The OS module in Python provides a way of using operating system dependent functionality. 
#The functions that the OS module provides allows you to interface with the underlying operating system 
#that Python is running on – be that Windows, Mac or Linux.

from os import listdir
from os.path import isfile, join

# Gensim
import gensim
import gensim.corpora as corpora
from gensim import models, corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

ModuleNotFoundError: No module named 'pyLDAvis'

In [2]:
import nltk as nltk 
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/christanasescu/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
stopwords = nltk.corpus.stopwords.words('stop_words_poetry.txt')

stopwords.append('...')
stopwords.append("'d")
stopwords.append('...')
stopwords.append("&")
stopwords.append("upon")
stopwords.append("also")
stopwords.append("hath")
stopwords.append("must")
stopwords.append("therefore")
stopwords.append("doth")
stopwords.append("could")
stopwords.append("would")
stopwords.append("another")
stopwords.append("much")
stopwords.append("give")
stopwords.append("like")
stopwords.append("since")
stopwords.append("many")
stopwords.append("without")
stopwords.append("first")
stopwords.append("though")
stopwords.append("well")
stopwords.append("often")
stopwords.append("great")
stopwords.append("either")
stopwords.append("even")
stopwords.append("shall")
stopwords.append("they")
stopwords.append("what")
stopwords.append("their")
stopwords.append("more")
stopwords.append("there")
stopwords.append("your")
stopwords.append("them")

In [4]:
def tokenize(text):
    tokens = word_tokenize(text)
    tokens = _pre_clean(tokens)
    tokens = [token for token in tokens if len(token) > 3]
    tokens = [token for token in tokens if token not in stopwords]
    #tokens = [get_lemma(token) for token in tokens]
    return tokens

In [5]:
pwd

'/Users/christanasescu'

In [6]:
import os

In [8]:
#cd fastText_multilingual-master

/Users/christanasescu/fastText_multilingual-master


In [7]:

HOME = os.getcwd()

In [8]:
def _pre_clean(list_of_text):
        '''
        preliminary cleaning of the text
        - remove new line character i.e. \n or \r
        - remove tabs i.e. \t
        - remove extra spaces
        '''
        cleaned_list = []
        for text in list_of_text:
            # print("original:", text)
            text = text.replace('\\n', ' ')
            text = text.replace('\\r', ' ')
            text = text.replace('\\t', ' ')
            pattern = re.compile(r'\s+')
            text = re.sub(pattern, ' ', text)
            text = text.strip()
            text = text.lower()
            # check for empty strings
            if text != '' and text is not None:
                cleaned_list.append(text)

        return cleaned_list

In [9]:
filelabels = {}

import re as re 

def get_documents(path):
    os.chdir(path)
    files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
    texts = []
    count = -1
    for f in files:
        with open(f, "r", encoding='utf-8', errors = 'ignore') as openf:
            count = count + 1
            filelabels[count] = os.path.basename(openf.name)
            splitted_lines = openf.read().splitlines()
            splitted_lines = _pre_clean(splitted_lines)
            texts.append(splitted_lines)
    print(filelabels)
    return texts

TEXTS_DIR = HOME + "/rushton_&_naum_for_margento_trans/"

documents = get_documents(TEXTS_DIR)

{0: 'corpus.pkl', 1: 'dictionary.gensim', 2: 'naum.txt', 3: 'rushton.txt'}


In [10]:

from nltk.tokenize import sent_tokenize, word_tokenize

In [11]:

texts_data = []

files = [f for f in os.listdir(TEXTS_DIR) if os.path.isfile(os.path.join(TEXTS_DIR, f))]

import string
from string import punctuation

remove_punct_map = dict.fromkeys(map(ord, string.punctuation))

tokens_total = []

for f in files:
    with open(f, "r", encoding='utf-8', errors = 'ignore') as openf:
        tokens = []
        for line in openf:
            sent_text = nltk.sent_tokenize(line)
            for sentence in sent_text:
                tokens1 = tokenize(sentence)
                tokens1 = [item.translate(remove_punct_map)
                      for item in tokens1]
                #filter_object = filter(lambda x: x != "", tokens1)
                tokens1 = [x for x in tokens1 if x!= ""]
                for token in tokens1:
                    tokens.append(token)
                    tokens_total.append(token)
    texts_data.append(tokens)


In [13]:

print(tokens_total) # apparently too large to print out here; see next 3 cells:

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [12]:

import pyperclip as clip

In [13]:

clip.copy(f"{tokens_total}")


Now we can simply do Command + V and thus copy-paste "tokens_total" into a txt file 

In [14]:

len(tokens_total)

8443


Seems that (some) stopwords still have not been flushed out. Therefore:

In [23]:

tokens_total = [x for x in tokens_total if x not in stopwords]

In [24]:

len(tokens_total)  # Yep, quite shorter:

8081

In [25]:
from collections import Counter

Count_total = Counter(tokens_total)
#print(Count_total)

In [19]:
clip.copy(f"{Count_total}")


Command+V into a page/word/txt file [or clip.paste() to print it here, but in this case it is too large a list to print]

In [26]:
print(Count_total)

Counter({'verse': 59, 'poet': 55, 'ovid': 49, 'when': 46, 'other': 41, 'time': 39, 'poem': 35, 'work': 32, 'know': 29, 'write': 26, 'naum': 25, 'poets': 25, 'said': 24, 'poetry': 23, 'long': 22, 'where': 22, 'still': 22, 'better': 22, 'writing': 21, 'lines': 21, 'marlowe': 21, 'down': 20, 'full': 20, 'night': 20, 'half': 20, 'under': 19, 'both': 19, 'over': 19, 'thought': 19, 'every': 19, 'dryden': 19, 'life': 18, 'between': 18, 'words': 18, 'line': 18, 'things': 17, 'sequence': 17, 'make': 17, 'might': 17, 'never': 16, 'before': 16, 'right': 16, 'maybe': 16, 'around': 16, 'water': 16, 'here': 16, 'love': 16, 'times': 16, 'after': 15, 'romanian': 15, 'made': 15, 'something': 15, 'course': 15, 'thanks': 15, 'best': 14, 'again': 14, 'word': 14, 'good': 14, 'each': 14, 'look': 14, 'came': 13, 'others': 13, 'found': 13, 'most': 13, 'take': 13, 'away': 13, 'eyes': 13, 'only': 13, 'ancient': 12, 'world': 12, 'went': 12, 'head': 12, 'perhaps': 12, 'three': 12, 'alter': 12, 'later': 11, 'death

In [27]:

for i in range(2):
    print(len(texts_data[i]))

4905
3538


In [28]:

for i in range(2):
    texts_data[i] = [x for x in texts_data[i] if x not in stopwords]

In [29]:

for i in range(2):
    print(len(texts_data[i]))  # Some of them got shorter:

4702
3379


In [30]:

dictionary = corpora.Dictionary(texts_data)

In [31]:
corpus = [dictionary.doc2bow(text) for text in texts_data]
import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [32]:
def compute_coherence_values(dictionary, corpus, texts, limit, start = 2, step = 1):
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.ldamodel.LdaModel(corpus = corpus, num_topics = num_topics, id2word = dictionary, random_state = 100, update_every = 1, passes = 15, alpha = 'auto', per_word_topics = True)
        model_list.append(model)
        coherencemodel = CoherenceModel(model = model, texts = texts, dictionary = dictionary, coherence = 'c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [33]:
model_list, coherence_values = compute_coherence_values(dictionary = dictionary, corpus = corpus, texts = texts_data, start=2, limit=10, step=1)

In [34]:
limit=10
start=2
step=1
x = range(start, limit, step)

for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

Num Topics = 2  has Coherence Value of 0.4488
Num Topics = 3  has Coherence Value of 0.3788
Num Topics = 4  has Coherence Value of 0.3508
Num Topics = 5  has Coherence Value of 0.4384
Num Topics = 6  has Coherence Value of 0.3979
Num Topics = 7  has Coherence Value of 0.4224
Num Topics = 8  has Coherence Value of 0.4113
Num Topics = 9  has Coherence Value of 0.359


In [35]:
NUM_TOPICS = 2
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, random_state=100, update_every=1, passes=15, alpha='auto', per_word_topics=True)



In [36]:
topics = ldamodel.print_topics(num_words=44)

In [28]:
#ldamodel.save('model5.gensim')

In [37]:
pprint(topics)

[(0,
  '0.011*"verse" + 0.009*"ovid" + 0.007*"poet" + 0.005*"work" + 0.005*"poem" + '
  '0.005*"write" + 0.004*"when" + 0.004*"time" + 0.004*"marlowe" + '
  '0.004*"poets" + 0.004*"lines" + 0.004*"other" + 0.004*"half" + '
  '0.004*"writing" + 0.004*"dryden" + 0.004*"said" + 0.004*"line" + '
  '0.003*"thought" + 0.003*"might" + 0.003*"every" + 0.003*"thanks" + '
  '0.003*"better" + 0.003*"poetry" + 0.003*"course" + 0.002*"alter" + '
  '0.002*"know" + 0.002*"three" + 0.002*"plan" + 0.002*"wine" + 0.002*"best" + '
  '0.002*"perhaps" + 0.002*"verses" + 0.002*"life" + 0.002*"before" + '
  '0.002*"make" + 0.002*"both" + 0.002*"between" + 0.002*"here" + '
  '0.002*"translation" + 0.002*"right" + 0.002*"take" + 0.002*"found" + '
  '0.002*"translations" + 0.002*"worse"'),
 (1,
  '0.004*"naum" + 0.004*"when" + 0.003*"other" + 0.003*"poet" + 0.003*"time" + '
  '0.003*"know" + 0.003*"sequence" + 0.003*"where" + 0.002*"long" + '
  '0.002*"water" + 0.002*"night" + 0.002*"still" + 0.002*"around" + '

In [ ]:
# We notice that the two topics roughly correpsond to the two collections, and therefore there is hardly any hybridization taking place. We need to consolidate the two files. 

In [38]:
cd ..

/Users/christanasescu


In [12]:
filelabels = {}

import re as re 

def get_documents(path):
    os.chdir(path)
    files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
    texts = []
    count = -1
    for f in files:
        with open(f, "r", encoding='utf-8', errors = 'ignore') as openf:
            count = count + 1
            filelabels[count] = os.path.basename(openf.name)
            splitted_lines = openf.read().splitlines()
            splitted_lines = _pre_clean(splitted_lines)
            texts.append(splitted_lines)
    print(filelabels)
    return texts

TEXTS_DIR = HOME + "/rushton_&_naum_consolidated_for_margento_trans/"

documents = get_documents(TEXTS_DIR)

{0: 'naum_&_rushton.txt'}


In [13]:
texts_data = []

files = [f for f in os.listdir(TEXTS_DIR) if os.path.isfile(os.path.join(TEXTS_DIR, f))]

import string
from string import punctuation

remove_punct_map = dict.fromkeys(map(ord, string.punctuation))

tokens_total = []

for f in files:
    with open(f, "r", encoding='utf-8', errors = 'ignore') as openf:
        tokens = []
        for line in openf:
            sent_text = nltk.sent_tokenize(line)
            for sentence in sent_text:
                tokens1 = tokenize(sentence)
                tokens1 = [item.translate(remove_punct_map)
                      for item in tokens1]
                #filter_object = filter(lambda x: x != "", tokens1)
                tokens1 = [x for x in tokens1 if x!= ""]
                for token in tokens1:
                    tokens.append(token)
                    tokens_total.append(token)
    texts_data.append(tokens)

In [41]:

len(tokens_total)

8082

In [14]:
tokens_total = [x for x in tokens_total if x not in stopwords]

In [43]:

len(tokens_total)

8081

In [44]:

Count_total = Counter(tokens_total)

In [45]:

print(Count_total)

Counter({'verse': 59, 'poet': 55, 'ovid': 49, 'when': 46, 'other': 41, 'time': 39, 'poem': 35, 'work': 32, 'know': 29, 'write': 26, 'poets': 25, 'naum': 25, 'said': 24, 'poetry': 23, 'long': 22, 'where': 22, 'better': 22, 'still': 22, 'writing': 21, 'marlowe': 21, 'lines': 21, 'night': 20, 'down': 20, 'half': 20, 'full': 20, 'dryden': 19, 'every': 19, 'thought': 19, 'both': 19, 'under': 19, 'over': 19, 'between': 18, 'line': 18, 'words': 18, 'life': 18, 'things': 17, 'might': 17, 'make': 17, 'sequence': 17, 'here': 16, 'love': 16, 'never': 16, 'times': 16, 'maybe': 16, 'right': 16, 'before': 16, 'around': 16, 'water': 16, 'thanks': 15, 'romanian': 15, 'after': 15, 'course': 15, 'something': 15, 'made': 15, 'each': 14, 'look': 14, 'best': 14, 'again': 14, 'word': 14, 'good': 14, 'only': 13, 'found': 13, 'away': 13, 'others': 13, 'came': 13, 'take': 13, 'most': 13, 'eyes': 13, 'alter': 12, 'head': 12, 'perhaps': 12, 'three': 12, 'ancient': 12, 'went': 12, 'world': 12, 'death': 11, 'later

In [46]:

for i in range(1):
    print(len(texts_data[i]))

8082


In [15]:

for i in range(1):
    texts_data[i] = [x for x in texts_data[i] if x not in stopwords]

In [48]:

print(len(texts_data[0]))

8081


In [16]:

dictionary = corpora.Dictionary(texts_data)

In [17]:

corpus = [dictionary.doc2bow(text) for text in texts_data]
import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [18]:

def compute_coherence_values(dictionary, corpus, texts, limit, start = 2, step = 1):
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.ldamodel.LdaModel(corpus = corpus, num_topics = num_topics, id2word = dictionary, random_state = 100, update_every = 1, passes = 15, alpha = 'auto', per_word_topics = True)
        model_list.append(model)
        coherencemodel = CoherenceModel(model = model, texts = texts, dictionary = dictionary, coherence = 'c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [19]:

model_list, coherence_values = compute_coherence_values(dictionary = dictionary, corpus = corpus, texts = texts_data, start=2, limit=10, step=1)

In [20]:

limit=10
start=2
step=1
x = range(start, limit, step)

for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

Num Topics = 2  has Coherence Value of 0.3536
Num Topics = 3  has Coherence Value of 0.3492
Num Topics = 4  has Coherence Value of 0.3415
Num Topics = 5  has Coherence Value of 0.3445
Num Topics = 6  has Coherence Value of 0.345
Num Topics = 7  has Coherence Value of 0.3281
Num Topics = 8  has Coherence Value of 0.3371
Num Topics = 9  has Coherence Value of 0.329


In [21]:

NUM_TOPICS = 2
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, random_state=100, update_every=1, passes=15, alpha='auto', per_word_topics=True)

In [22]:

topics = ldamodel.print_topics(num_words=44)

In [56]:

pprint(topics)

[(0,
  '0.006*"verse" + 0.006*"poet" + 0.005*"ovid" + 0.005*"when" + 0.004*"other" '
  '+ 0.004*"time" + 0.004*"poem" + 0.003*"work" + 0.003*"know" + 0.003*"write" '
  '+ 0.003*"poets" + 0.003*"naum" + 0.002*"said" + 0.002*"poetry" + '
  '0.002*"long" + 0.002*"where" + 0.002*"still" + 0.002*"better" + '
  '0.002*"lines" + 0.002*"marlowe" + 0.002*"writing" + 0.002*"down" + '
  '0.002*"night" + 0.002*"half" + 0.002*"full" + 0.002*"over" + 0.002*"every" '
  '+ 0.002*"both" + 0.002*"thought" + 0.002*"under" + 0.002*"dryden" + '
  '0.002*"life" + 0.002*"line" + 0.002*"words" + 0.002*"between" + '
  '0.002*"sequence" + 0.002*"things" + 0.002*"might" + 0.002*"make" + '
  '0.002*"right" + 0.002*"before" + 0.002*"here" + 0.002*"around" + '
  '0.002*"maybe"'),
 (1,
  '0.000*"verse" + 0.000*"poet" + 0.000*"other" + 0.000*"ovid" + 0.000*"time" '
  '+ 0.000*"work" + 0.000*"when" + 0.000*"poem" + 0.000*"naum" + 0.000*"poets" '
  '+ 0.000*"write" + 0.000*"said" + 0.000*"know" + 0.000*"better" + '
  '

In [57]:

ldamodel.get_term_topics("master", minimum_probability = 0)

[(0, 0.0005021314), (1, 7.6084754e-05)]

In [24]:

l0 = list(ldamodel.show_topic(0, topn = 9000))

In [25]:
l1 = list(ldamodel.show_topic(1, topn = 9000))

In [61]:

for i in range(9000):
    if l0[i][0]== "master":
            print(i)

300


IndexError: list index out of range

In [62]:
for i in range(9000):
    if l1[i][0]== "master":
            print(i)

235


IndexError: list index out of range

In [63]:

for i in range(280, 320):
    print(l0[i])

('blood', 0.00055153354)
('based', 0.00055152853)
('against', 0.0005515245)
('known', 0.00055152446)
('looked', 0.00055152044)
('romantic', 0.00055151456)
('distance', 0.00055151386)
('dream', 0.00055150955)
('green', 0.0005515067)
('london', 0.00055150496)
('theme', 0.0005515028)
('cealaltă', 0.0005515024)
('published', 0.00055149966)
('wait', 0.0005514934)
('shoulder', 0.0005514927)
('anyway', 0.0005514926)
('despair', 0.0005514893)
('telling', 0.00055148697)
('pass', 0.0005514781)
('fața', 0.00055147696)
('master', 0.00055147626)
('myself', 0.0005514739)
('rental', 0.0005514735)
('lance', 0.00055146875)
('vintage', 0.0005514583)
('thus', 0.00055145036)
('council', 0.0005514432)
('especially', 0.00055144203)
('girls', 0.000551435)
('happy', 0.00055143284)
('partea', 0.0005514313)
('finger', 0.00055142667)
('passed', 0.0005514195)
('ashes', 0.00055140705)
('painting', 0.0005513837)
('little', 0.0005513822)
('mind', 0.0005513486)
('metamorphoses', 0.0005513394)
('fourfour', 0.000551337

In [64]:

for i in range(215, 255):
    print(l1[i])

('beloved', 0.00026639234)
('happy', 0.0002663874)
('drowned', 0.00026638736)
('girls', 0.00026637627)
('people', 0.00026634586)
('especially', 0.00026633995)
('council', 0.00026633334)
('thus', 0.0002662956)
('iron', 0.00026626023)
('vintage', 0.0002662547)
('everybody', 0.00026624987)
('hands', 0.000266245)
('begin', 0.0002662189)
('trees', 0.00026621282)
('rome', 0.00026620715)
('lance', 0.00026620002)
('1968', 0.00026617976)
('group', 0.00026617598)
('rental', 0.0002661749)
('myself', 0.0002661731)
('master', 0.00026616117)
('fața', 0.0002661572)
('inside', 0.00026615415)
('pass', 0.00026615092)
('exile', 0.0002661433)
('across', 0.00026613422)
('perfect', 0.00026612086)
('telling', 0.0002661047)
('despair', 0.0002660927)
('interest', 0.0002660859)
('outside', 0.0002660858)
('anyway', 0.00026607575)
('shoulder', 0.0002660751)
('feel', 0.00026607205)
('wait', 0.00026607118)
('secret', 0.00026606736)
('published', 0.0002660383)
('story', 0.00026603375)
('cealaltă', 0.000266024)
('nee

In [66]:

len(l0)

3785

In [68]:

len(l1)

3785

In [67]:

for i in range(3785):
    if l0[i][0]== "project":
            print(i)

258


In [69]:

for i in range(3785):
    if l1[i][0]== "project":
            print(i)

313


In [71]:

for i in range(237, 278):
    print(l0[i])

('lips', 0.0005516778)
('asked', 0.0005516723)
('memories', 0.0005516694)
('waiting', 0.00055166177)
('zenobia', 0.0005516551)
('poems', 0.00055163365)
('language', 0.00055162713)
('bird', 0.0005516147)
('case', 0.00055161444)
('illustrations', 0.00055161404)
('rock', 0.00055161363)
('stop', 0.0005516103)
('poor', 0.00055160985)
('ocean', 0.0005516083)
('triangle', 0.0005516026)
('minor', 0.0005516012)
('room', 0.0005516006)
('tangled', 0.0005515951)
('enough', 0.0005515893)
('foot', 0.0005515845)
('discussion', 0.0005515798)
('project', 0.0005515763)
('derby', 0.00055157003)
('next', 0.0005515693)
('fact', 0.0005515688)
('rhyme', 0.00055156834)
('past', 0.0005515683)
('albastru', 0.0005515652)
('become', 0.0005515641)
('alchemical', 0.0005515623)
('exiled', 0.0005515599)
('sound', 0.00055155763)
('marvellous', 0.0005515529)
('eutychia', 0.00055155216)
('malul', 0.00055155176)
('mist', 0.000551547)
('artists', 0.00055154663)
('loved', 0.0005515449)
('meaning', 0.0005515447)
('toward', 

In [72]:

for i in range(292, 333):
    print(l1[i])

('marvellous', 0.00026575974)
('brought', 0.000265744)
('sound', 0.0002657362)
('terrible', 0.00026572982)
('exiled', 0.00026572254)
('alchemical', 0.00026571116)
('become', 0.0002657005)
('albastru', 0.0002656963)
('stand', 0.00026569597)
('wells', 0.00026569248)
('drunken', 0.00026568625)
('lying', 0.0002656811)
('past', 0.00026567996)
('rhyme', 0.00026567932)
('fact', 0.00026567714)
('next', 0.00026567478)
('camera', 0.00026567274)
('derby', 0.00026567033)
('urmat', 0.00026566963)
('leaves', 0.00026566055)
('saying', 0.00026564513)
('project', 0.00026563765)
('discussion', 0.00026561992)
('beginning', 0.00026561224)
('round', 0.00026560697)
('foot', 0.00026559463)
('describing', 0.000265594)
('sure', 0.0002655859)
('listening', 0.00026558276)
('1994', 0.0002655821)
('certain', 0.00026558017)
('communist', 0.0002655782)
('explanation', 0.00026557798)
('motioned', 0.00026557714)
('enough', 0.00026556858)
('higher', 0.00026556174)
('reasons', 0.00026555968)
('beneath', 0.00026555016)
(

In [74]:

for i in range(3785):
    if l0[i][0]== "contented":
            print(i)

2517


In [75]:
for i in range(3785):
    if l1[i][0]== "contented":
            print(i)

2559


In [77]:

for i in range(2496, 2536):
    print(l0[i])

('colorfully', 0.0001503739)
('loads', 0.00015037389)
('moods', 0.00015037387)
('route', 0.00015037387)
('bottles—drinks', 0.00015037385)
('poverty—well', 0.0001503738)
('butterflies', 0.0001503738)
('nightingales', 0.0001503738)
('incompetent', 0.00015037374)
('grace', 0.00015037374)
('appealing', 0.00015037374)
('talks', 0.00015037373)
('worry', 0.00015037373)
('brushed', 0.0001503737)
('wondrously', 0.0001503737)
('esperanza', 0.00015037367)
('hunger', 0.00015037366)
('grave', 0.00015037366)
('scan', 0.00015037364)
('acts', 0.00015037364)
('converse', 0.0001503736)
('contented', 0.00015037358)
('alexandrian', 0.00015037357)
('telephoto', 0.00015037355)
('reciter', 0.00015037355)
('students', 0.00015037355)
('wrinkles', 0.00015037353)
('rochester', 0.0001503735)
('horizon', 0.00015037347)
('unfaltering', 0.00015037347)
('clocks', 0.00015037347)
('done—', 0.00015037345)
('interiorexterior', 0.00015037344)
('stanza', 0.00015037342)
('către', 0.00015037341)
('fumbling', 0.0001503734)
('

In [78]:

for i in range(2537, 2579):
    print(l1[i])

('darkyears', 0.00026352683)
('discouraging', 0.00026352674)
('besiege', 0.00026352666)
('either—', 0.00026352637)
('astral', 0.00026352634)
('lamp', 0.00026352625)
('nobility', 0.0002635262)
('carefully', 0.00026352602)
('ferry', 0.00026352596)
('crying', 0.00026352578)
('drank', 0.00026352567)
('amount', 0.00026352555)
('fumbling', 0.00026352552)
('stanza', 0.0002635255)
('done—', 0.0002635255)
('către', 0.0002635254)
('interiorexterior', 0.00026352538)
('horizon', 0.00026352535)
('unfaltering', 0.00026352535)
('clocks', 0.00026352514)
('wrinkles', 0.000263525)
('rochester', 0.0002635249)
('contented', 0.0002635248)
('reciter', 0.0002635248)
('telephoto', 0.00026352477)
('converse', 0.00026352468)
('alexandrian', 0.00026352462)
('students', 0.0002635246)
('grave', 0.00026352436)
('hunger', 0.00026352433)
('scan', 0.0002635243)
('acts', 0.00026352415)
('esperanza', 0.0002635241)
('wondrously', 0.0002635241)
('brushed', 0.00026352392)
('incompetent', 0.0002635239)
('worry', 0.000263523

In [79]:

for i in range(3785):
    if l0[i][0]== "dross":
            print(i)

In [80]:

for i in range(3785):
    if l1[i][0]== "dross":
            print(i)

In [81]:

for i in range(3785):
    if l0[i][0]== "salvaged":
            print(i)

2830


In [82]:

for i in range(3785):
    if l0[i][0]== "part":
            print(i)

439


In [83]:

for i in range(3785):
    if l1[i][0]== "part":
            print(i)

287


In [84]:

for i in range(418, 460):
    print(l0[i])

('higher', 0.00045118804)
('explanation', 0.00045118507)
('motioned', 0.000451185)
('communist', 0.00045118487)
('certain', 0.00045118458)
('listening', 0.00045118429)
('1994', 0.00045118414)
('sure', 0.00045118353)
('describing', 0.00045118184)
('beginning', 0.0004511782)
('saying', 0.000451172)
('leaves', 0.00045116918)
('urmat', 0.00045116735)
('camera', 0.00045116688)
('lying', 0.00045116528)
('drunken', 0.0004511643)
('wells', 0.00045116319)
('stand', 0.00045116252)
('terrible', 0.000451156)
('brought', 0.00045115323)
('brauner', 0.0004511471)
('part', 0.00045114546)
('evening', 0.00045114412)
('gallery', 0.00045112695)
('becomes', 0.00045112555)
('fisherman', 0.00045110067)
('needed', 0.00045109974)
('interest', 0.0004510877)
('theater', 0.0003510728)
('scarf', 0.00035106906)
('return', 0.00035106234)
('plot', 0.00035105442)
('very', 0.00035104842)
('appendices', 0.00035104097)
('subtle', 0.00035103934)
('concepts', 0.00035103917)
('despite', 0.00035103902)
('void', 0.00035103114

In [85]:

for i in range(266, 307):
    print(l1[i])

('english', 0.0002659175)
('against', 0.00026590808)
('known', 0.00026590808)
('becomes', 0.00026588884)
('based', 0.00026588768)
('breasts', 0.0002658864)
('gallery', 0.00026588168)
('bucharest', 0.00026587278)
('blood', 0.00026586137)
('till', 0.00026585895)
('state', 0.00026584222)
('constanta', 0.0002658393)
('style', 0.00026583666)
('girl', 0.00026583372)
('stone', 0.0002658113)
('toward', 0.00026580394)
('loved', 0.00026580266)
('meaning', 0.00026580234)
('artists', 0.00026579318)
('evening', 0.00026579187)
('mist', 0.00026579064)
('part', 0.00026578474)
('waters', 0.0002657821)
('brauner', 0.000265776)
('malul', 0.00026576564)
('eutychia', 0.0002657644)
('marvellous', 0.00026575974)
('brought', 0.000265744)
('sound', 0.0002657362)
('terrible', 0.00026572982)
('exiled', 0.00026572254)
('alchemical', 0.00026571116)
('become', 0.0002657005)
('albastru', 0.0002656963)
('stand', 0.00026569597)
('wells', 0.00026569248)
('drunken', 0.00026568625)
('lying', 0.0002656811)
('past', 0.0002

In [86]:

for i in range(3785):
    if l0[i][0]== "food":
            print(i)

707


In [87]:

for i in range(3785):
    if l1[i][0]== "food":
            print(i)

1311


In [88]:

for i in range(686, 736):
    print(l0[i])

('symbol', 0.00035086388)
('british', 0.0003508634)
('bottle', 0.00035086164)
('moment', 0.00035085852)
('diva', 0.0003508572)
('smelled', 0.00035085395)
('renaissance', 0.00035085384)
('willows', 0.0003508468)
('works', 0.00035084572)
('enjoying', 0.0003508182)
('lygia', 0.00035081457)
('descrierea', 0.00035081405)
('takes', 0.00035079807)
('american', 0.0002507637)
('incredible', 0.00025074606)
('wheels', 0.00025074455)
('bearer', 0.0002507409)
('held', 0.00025074056)
('favourite', 0.0002507385)
('rifle', 0.00025073625)
('deeply', 0.00025073488)
('food', 0.00025073462)
('fast', 0.00025073337)
('blanket', 0.00025073238)
('migrations', 0.0002507312)
('close', 0.00025073107)
('edited', 0.0002507301)
('wearing', 0.0002507249)
('tunes', 0.00025072432)
('artistic', 0.0002507243)
('whoever', 0.00025072362)
('inspiring', 0.00025072202)
('terms', 0.00025072176)
('whereas', 0.0002507208)
('repetition', 0.0002507172)
('staid', 0.0002507151)
('fruit', 0.00025071492)
('approached', 0.00025071454)

In [89]:

for i in range(1289, 1331):
    print(l1[i])

('assured', 0.00026377087)
('perpetual', 0.000263765)
('lively', 0.00026375666)
('edited', 0.0002637507)
('translatingwriting', 0.0002637501)
('dyed', 0.00026374782)
('close', 0.00026374563)
('migrations', 0.00026374546)
('realm', 0.00026374465)
('ascet', 0.00026374342)
('linen', 0.00026374112)
('blanket', 0.00026373914)
('king', 0.0002637362)
('bone', 0.00026373402)
('fast', 0.00026373393)
('songs', 0.00026373347)
('heine', 0.0002637333)
('flares', 0.00026373222)
('bounty', 0.00026373018)
('tones', 0.00026372992)
('cast', 0.00026372873)
('flooded', 0.0002637287)
('food', 0.0002637273)
('contain', 0.0002637268)
('deeply', 0.00026372581)
('informing', 0.00026372354)
('ceremonies', 0.00026372142)
('rifle', 0.00026371935)
('alchemist', 0.000263719)
('schimbarea', 0.00026371705)
('smelling', 0.000263717)
('foremost', 0.00026371563)
('conservation', 0.00026371493)
('editing', 0.00026371493)
('degas', 0.00026371438)
('rustling', 0.00026371362)
('deserves', 0.00026371283)
('cages', 0.00026371

In [90]:

for i in range(3785):
    if l0[i][0]== "read":
            print(i)

115


In [91]:

for i in range(3785):
    if l1[i][0]== "read":
            print(i)

83


In [92]:

for i in range(94, 136):
    print(l0[i])

('place', 0.0010529556)
('verses', 0.0010528893)
('earth', 0.0010528528)
('form', 0.0010528367)
('silence', 0.0010526759)
('rhythm', 0.0009528642)
('always', 0.0009528486)
('least', 0.00095284573)
('window', 0.00095283013)
('keep', 0.0009528049)
('lost', 0.00095278886)
('worse', 0.0009527521)
('hand', 0.00095270935)
('different', 0.0009527035)
('change', 0.00095268065)
('born', 0.00095267536)
('waited', 0.0009526636)
('beautiful', 0.0009526461)
('tree', 0.0009525278)
('left', 0.00095251354)
('translations', 0.0009525107)
('read', 0.0009525102)
('coming', 0.00095246214)
('athanor', 0.00095243903)
('beloved', 0.0008526401)
('quiet', 0.00085263466)
('once', 0.0008526234)
('pillars', 0.00085261767)
('someone', 0.0008526091)
('tell', 0.00085249037)
('making', 0.00085248885)
('less', 0.00085248874)
('think', 0.00085248245)
('grass', 0.0008524459)
('problem', 0.0008524448)
('glass', 0.00085244013)
('same', 0.0008524378)
('amores', 0.0008524238)
('together', 0.00085241254)
('feet', 0.000852391

In [93]:

for i in range(52, 103):
    print(l1[i])

('side', 0.00027153018)
('found', 0.00027146054)
('might', 0.00027138935)
('things', 0.00027132957)
('after', 0.00027112634)
('sequence', 0.00027107768)
('alter', 0.00027100928)
('word', 0.00027074935)
('before', 0.0002707399)
('says', 0.00027062916)
('most', 0.00027060544)
('ancient', 0.00027056565)
('blue', 0.0002705586)
('eyes', 0.00027043445)
('silence', 0.00027041862)
('made', 0.00027034653)
('romanian', 0.00027030715)
('right', 0.00027020922)
('perhaps', 0.0002702087)
('later', 0.0002701604)
('away', 0.00027006742)
('course', 0.00026999714)
('good', 0.00026959524)
('form', 0.000269577)
('athanor', 0.00026954978)
('wine', 0.0002695088)
('earth', 0.00026948933)
('coming', 0.0002694289)
('others', 0.00026932464)
('plan', 0.00026930892)
('verses', 0.00026930167)
('read', 0.00026917702)
('translations', 0.00026917487)
('left', 0.00026916078)
('world', 0.0002691376)
('tree', 0.0002690837)
('head', 0.00026906483)
('best', 0.00026905094)
('went', 0.00026900566)
('grow', 0.00026897786)
('

In [97]:

for i in range(3785):
    if l0[i][0]== "verses":
            print(i)

95


In [98]:
for i in range(3785):
    if l1[i][0]== "verses":
            print(i)

82


In [100]:

for i in range(73, 115):
    print(l0[i])


('alter', 0.0012533682)
('three', 0.0012531913)
('above', 0.001153581)
('death', 0.0011534953)
('face', 0.0011533841)
('plan', 0.0011532906)
('wine', 0.0011532522)
('later', 0.0011531275)
('blue', 0.0011530515)
('says', 0.001153038)
('side', 0.0011528659)
('translation', 0.0010532183)
('until', 0.0010532007)
('started', 0.001053197)
('black', 0.0010530917)
('find', 0.0010530588)
('light', 0.0010530384)
('moon', 0.0010530237)
('second', 0.0010530044)
('come', 0.0010529625)
('everything', 0.0010529559)
('place', 0.0010529556)
('verses', 0.0010528893)
('earth', 0.0010528528)
('form', 0.0010528367)
('silence', 0.0010526759)
('rhythm', 0.0009528642)
('always', 0.0009528486)
('least', 0.00095284573)
('window', 0.00095283013)
('keep', 0.0009528049)
('lost', 0.00095278886)
('worse', 0.0009527521)
('hand', 0.00095270935)
('different', 0.0009527035)
('change', 0.00095268065)
('born', 0.00095267536)
('waited', 0.0009526636)
('beautiful', 0.0009526461)
('tree', 0.0009525278)
('left', 0.00095251354

In [99]:
for i in range(61, 105):
    print(l1[i])

('says', 0.00027062916)
('most', 0.00027060544)
('ancient', 0.00027056565)
('blue', 0.0002705586)
('eyes', 0.00027043445)
('silence', 0.00027041862)
('made', 0.00027034653)
('romanian', 0.00027030715)
('right', 0.00027020922)
('perhaps', 0.0002702087)
('later', 0.0002701604)
('away', 0.00027006742)
('course', 0.00026999714)
('good', 0.00026959524)
('form', 0.000269577)
('athanor', 0.00026954978)
('wine', 0.0002695088)
('earth', 0.00026948933)
('coming', 0.0002694289)
('others', 0.00026932464)
('plan', 0.00026930892)
('verses', 0.00026930167)
('read', 0.00026917702)
('translations', 0.00026917487)
('left', 0.00026916078)
('world', 0.0002691376)
('tree', 0.0002690837)
('head', 0.00026906483)
('best', 0.00026905094)
('went', 0.00026900566)
('grow', 0.00026897786)
('five', 0.000268956)
('place', 0.00026895374)
('everything', 0.00026895298)
('come', 0.0002689184)
('face', 0.00026881965)
('need', 0.0002687301)
('second', 0.00026870024)
('moon', 0.0002685978)
('years', 0.00026853388)
('light'

In [101]:

for i in range(3785):
    if l0[i][0]== "garbage":
            print(i)

3114


In [102]:

for i in range(3785):
    if l1[i][0]== "garbage":
            print(i)

1964


In [104]:

for i in range(3093, 3136):
    print(l0[i])

('sails', 0.00015036379)
('banalities', 0.00015036376)
('fountains', 0.00015036372)
('embeds', 0.00015036369)
('eight', 0.00015036367)
('shivering', 0.00015036364)
('studio', 0.0001503636)
('disposing', 0.00015036359)
('london—marlowe', 0.00015036353)
('cooked', 0.00015036351)
('forgets', 0.00015036348)
('refused', 0.00015036348)
('messy', 0.00015036346)
('presley', 0.00015036346)
('orange', 0.00015036346)
('square', 0.00015036343)
('seat', 0.00015036343)
('manes', 0.00015036341)
('academic', 0.00015036341)
('finest', 0.00015036341)
('triple', 0.00015036338)
('garbage', 0.00015036337)
('circling', 0.00015036335)
('secluded', 0.00015036335)
('number', 0.00015036335)
('cultures', 0.00015036334)
('prepared', 0.00015036328)
('intimately', 0.00015036327)
('wobbling', 0.00015036324)
('warner', 0.00015036324)
('current', 0.00015036322)
('reef', 0.00015036316)
('couplet', 0.00015036316)
('keys—the', 0.00015036316)
('grape', 0.00015036315)
('postcards', 0.00015036309)
('pietre', 0.00015036308)


In [103]:

for i in range(1943, 1985):
    print(l1[i])

('neutral', 0.00026357995)
('clues', 0.00026357986)
('saucy', 0.00026357966)
('chairs', 0.0002635794)
('selling', 0.00026357928)
('pietre', 0.00026357916)
('burns', 0.0002635791)
('postcards', 0.0002635791)
('dislocations—not', 0.000263579)
('keys—the', 0.0002635788)
('grape', 0.0002635788)
('reef', 0.00026357875)
('couplet', 0.0002635786)
('current', 0.00026357843)
('warner', 0.0002635783)
('wobbling', 0.0002635782)
('intimately', 0.00026357817)
('prepared', 0.0002635779)
('cultures', 0.00026357788)
('secluded', 0.0002635778)
('circling', 0.00026357776)
('garbage', 0.00026357762)
('number', 0.00026357747)
('manes', 0.00026357747)
('triple', 0.00026357747)
('finest', 0.0002635774)
('seat', 0.00026357727)
('messy', 0.00026357727)
('presley', 0.00026357727)
('orange', 0.0002635772)
('square', 0.00026357718)
('academic', 0.00026357715)
('london—marlowe', 0.00026357707)
('refused', 0.000263577)
('forgets', 0.00026357695)
('cooked', 0.00026357686)
('studio', 0.0002635765)
('disposing', 0.00

In [105]:

for i in range(3785):
    if l0[i][0]== "fruition":
            print(i)

2893


In [106]:

for i in range(3785):
    if l1[i][0]== "fruition":
            print(i)

2186


In [107]:

for i in range(2871, 2906):
    print(l0[i])

('motorbike', 0.00015036788)
('communicate', 0.00015036786)
('20th', 0.0001503678)
('tension', 0.00015036779)
('swear', 0.00015036779)
('reworking', 0.00015036775)
('college', 0.00015036773)
('fluidity', 0.0001503677)
('relation', 0.00015036769)
('notably', 0.00015036769)
('staring', 0.00015036769)
('continuously', 0.00015036766)
('discurs', 0.00015036766)
('creaked', 0.00015036766)
('vague', 0.00015036763)
('password', 0.00015036754)
('finale', 0.0001503675)
('extinct', 0.00015036747)
('bores', 0.00015036747)
('received', 0.00015036746)
('pohem', 0.00015036746)
('affray', 0.00015036741)
('fruition', 0.00015036738)
('adventurous', 0.00015036736)
('inducing', 0.00015036736)
('shrouds', 0.00015036736)
('supervisor', 0.00015036736)
('heyday—ruins', 0.00015036733)
('dispute', 0.00015036733)
('represents', 0.00015036731)
('feel—and', 0.00015036731)
('obviously', 0.00015036731)
('tells', 0.00015036728)
('careless', 0.00015036728)
('salute', 0.00015036725)


In [108]:

for i in range(2163, 2210):
    print(l1[i])

('xxxiv', 0.00026355838)
('send', 0.00026355832)
('erudition', 0.0002635582)
('xlii', 0.0002635582)
('decrease', 0.00026355791)
('rows', 0.00026355783)
('traveler', 0.00026355774)
('sarane', 0.0002635577)
('shallows', 0.00026355768)
('code', 0.00026355768)
('crossed', 0.0002635576)
('salute', 0.00026355754)
('cycle', 0.0002635575)
('feel—and', 0.00026355725)
('careless', 0.0002635572)
('represents', 0.00026355716)
('obviously', 0.00026355713)
('dispute', 0.0002635571)
('inducing', 0.00026355704)
('tells', 0.000263557)
('adventurous', 0.00026355698)
('supervisor', 0.00026355698)
('heyday—ruins', 0.00026355695)
('fruition', 0.00026355675)
('affray', 0.0002635567)
('shrouds', 0.0002635567)
('pohem', 0.00026355663)
('received', 0.0002635564)
('bores', 0.0002635564)
('extinct', 0.00026355634)
('finale', 0.00026355614)
('password', 0.0002635559)
('vague', 0.00026355564)
('creaked', 0.0002635553)
('discurs', 0.0002635553)
('notably', 0.0002635553)
('staring', 0.0002635553)
('continuously', 0.

In [109]:

for i in range(3785):
    if l0[i][0]== "disruptions":
            print(i)

3471


In [110]:

for i in range(3785):
    if l1[i][0]== "disruptions":
            print(i)

1608


In [111]:

for i in range(3450, 3491):
    print(l0[i])

('mid1930s', 0.00015035542)
('work—poetry', 0.00015035536)
('alley', 0.00015035532)
('divergent', 0.00015035529)
('doglike', 0.00015035529)
('differentiating', 0.00015035529)
('sources', 0.00015035522)
('wounds', 0.00015035515)
('peril', 0.00015035512)
('complexities', 0.00015035512)
('laundry', 0.00015035506)
('result', 0.00015035504)
('goals', 0.000150355)
('light—', 0.000150355)
('bolero', 0.00015035497)
('climate', 0.00015035496)
('secretă', 0.0001503549)
('uses', 0.0001503549)
('occluded', 0.00015035488)
('sang', 0.00015035486)
('settled', 0.00015035483)
('disruptions', 0.00015035481)
('impotence', 0.00015035478)
('deserve', 0.00015035478)
('perished', 0.00015035478)
('found—and', 0.00015035475)
('infinitely', 0.00015035472)
('funded', 0.00015035465)
('popescu', 0.00015035464)
('disposed', 0.0001503545)
('occasion—when', 0.00015035446)
('complications', 0.00015035443)
('italy—', 0.00015035443)
('rescue—an', 0.00015035442)
('rhyming', 0.0001503544)
('seducing', 0.00015035438)
('dwe

In [112]:

for i in range(1588, 1630):
    print(l1[i])

('revisioned', 0.00026362488)
('versions', 0.00026362465)
('striking', 0.0002636246)
('gazes', 0.00026362445)
('dwellers', 0.0002636242)
('amulets', 0.0002636242)
('seducing', 0.00026362418)
('rhyming', 0.00026362416)
('rescue—an', 0.00026362416)
('italy—', 0.00026362413)
('complications', 0.000263624)
('occasion—when', 0.00026362372)
('disposed', 0.00026362334)
('popescu', 0.0002636229)
('funded', 0.00026362285)
('infinitely', 0.00026362255)
('found—and', 0.00026362235)
('impotence', 0.0002636222)
('perished', 0.00026362218)
('deserve', 0.00026362215)
('disruptions', 0.00026362203)
('settled', 0.00026362194)
('sang', 0.0002636218)
('occluded', 0.00026362165)
('secretă', 0.00026362156)
('uses', 0.00026362154)
('climate', 0.00026362127)
('bolero', 0.00026362124)
('goals', 0.0002636211)
('result', 0.00026362107)
('laundry', 0.00026362078)
('light—', 0.00026362075)
('peril', 0.00026362043)
('complexities', 0.0002636202)
('wounds', 0.00026362017)
('sources', 0.00026362008)
('differentiatin

In [113]:

for i in range(3785):
    if l0[i][0]== "etymology":
            print(i)

2462


In [114]:

for i in range(3785):
    if l1[i][0]== "etymology":
            print(i)

2624


In [117]:

for i in range(2442, 2483):
    print(l0[i])

('mumble', 0.0001503749)
('persistently', 0.0001503748)
('centred', 0.00015037479)
('mixing', 0.00015037476)
('reminiscent', 0.00015037476)
('extended', 0.00015037475)
('weather', 0.00015037473)
('solution', 0.0001503747)
('grate', 0.00015037468)
('minor—', 0.00015037463)
('many—even', 0.00015037463)
('adapt', 0.00015037463)
('rambling', 0.00015037459)
('dancers', 0.00015037452)
('fragmentation', 0.0001503745)
('picture', 0.0001503745)
('practice', 0.0001503745)
('acknowledgement', 0.00015037449)
('deaf', 0.00015037449)
('greatest', 0.00015037447)
('etymology', 0.00015037447)
('curtains', 0.00015037444)
('extinguished', 0.00015037443)
('surely', 0.00015037443)
('fallen', 0.00015037438)
('huge', 0.00015037436)
('smokes', 0.00015037434)
('sevenbeat', 0.00015037434)
('warriors', 0.00015037433)
('intently', 0.00015037425)
('understand', 0.00015037425)
('gusto', 0.00015037425)
('focusing', 0.00015037425)
('ranks', 0.00015037425)
('miss', 0.00015037425)
('studied', 0.00015037422)
('borsht', 

In [118]:

for i in range(2603, 2664):
    print(l1[i])

('understand', 0.0002635213)
('intently', 0.00026352127)
('ranks', 0.00026352124)
('borsht', 0.00026352124)
('miss', 0.00026352107)
('focusing', 0.00026352104)
('gusto', 0.00026352095)
('warriors', 0.00026352087)
('sevenbeat', 0.00026352084)
('smokes', 0.00026352075)
('huge', 0.00026352057)
('fallen', 0.00026352055)
('surely', 0.00026352017)
('curtains', 0.00026352008)
('greatest', 0.00026352002)
('deaf', 0.00026351996)
('extinguished', 0.00026351993)
('practice', 0.00026351993)
('acknowledgement', 0.00026351993)
('fragmentation', 0.00026351988)
('picture', 0.00026351985)
('etymology', 0.00026351982)
('dancers', 0.00026351973)
('rambling', 0.0002635193)
('minor—', 0.00026351915)
('adapt', 0.000263519)
('many—even', 0.00026351894)
('grate', 0.0002635189)
('solution', 0.00026351874)
('weather', 0.00026351865)
('mixing', 0.00026351857)
('extended', 0.00026351854)
('centred', 0.00026351848)
('reminiscent', 0.00026351822)
('persistently', 0.0002635182)
('mumble', 0.0002635178)
('lurking', 0

In [119]:

for i in range(3785):
    if l0[i][0]== "locus":
            print(i)

2651


In [121]:

for i in range(3785):
    if l1[i][0]== "locus":
            print(i)

2429


In [123]:

for i in range(2630, 2673):
    print(l0[i])

('shrubs', 0.00015037185)
('bypassing', 0.00015037184)
('wallachian', 0.00015037184)
('anticipating', 0.00015037181)
('collector', 0.00015037181)
('shifted', 0.00015037181)
('struggles', 0.00015037176)
('blunt', 0.00015037175)
('labyrinth', 0.00015037174)
('ordered', 0.00015037174)
('pilgrimage', 0.00015037172)
('tireless', 0.00015037172)
('democracy', 0.00015037172)
('reunited', 0.0001503717)
('postcard', 0.0001503717)
('enforced', 0.0001503717)
('backing', 0.00015037165)
('influenced', 0.00015037165)
('—protagonists', 0.00015037159)
('woes', 0.00015037156)
('destructive', 0.00015037153)
('locus', 0.0001503715)
('xxxii', 0.00015037137)
('thrown', 0.00015037137)
('elena', 0.00015037134)
('categorical', 0.00015037133)
('knock', 0.00015037133)
('arranged', 0.0001503713)
('indecent', 0.00015037126)
('pohetry', 0.00015037121)
('dahlias', 0.0001503712)
('audience', 0.00015037118)
('retreat', 0.00015037117)
('incontinence', 0.00015037117)
('worked', 0.0001503711)
('developing', 0.00015037107

In [122]:

for i in range(2408, 2449):
    print(l1[i])

('discerned', 0.0002635386)
('lord', 0.0002635385)
('calling', 0.00026353847)
('brazen', 0.00026353827)
('timpul', 0.00026353824)
('unhumorous', 0.00026353824)
('balance', 0.00026353795)
('developing', 0.0002635376)
('worked', 0.00026353748)
('incontinence', 0.00026353722)
('retreat', 0.00026353716)
('dahlias', 0.00026353713)
('pohetry', 0.00026353705)
('audience', 0.00026353693)
('indecent', 0.00026353658)
('arranged', 0.00026353652)
('thrown', 0.00026353635)
('elena', 0.00026353626)
('knock', 0.00026353623)
('categorical', 0.0002635362)
('xxxii', 0.00026353612)
('locus', 0.00026353533)
('woes', 0.0002635351)
('destructive', 0.0002635351)
('—protagonists', 0.000263535)
('backing', 0.0002635347)
('reunited', 0.00026353446)
('influenced', 0.00026353443)
('tireless', 0.0002635344)
('democracy', 0.00026353437)
('pilgrimage', 0.0002635342)
('enforced', 0.0002635342)
('blunt', 0.00026353414)
('postcard', 0.00026353414)
('labyrinth', 0.00026353405)
('ordered', 0.00026353405)
('struggles', 0.

In [124]:

for i in range(3785):
    if l0[i][0]== "wild":
            print(i)

835


In [125]:

for i in range(3785):
    if l1[i][0]== "wild":
            print(i)

1159


In [127]:

for i in range(814, 866):
    print(l0[i])

('feathers', 0.0002506887)
('vision', 0.0002506886)
('heavy', 0.0002506881)
('empty', 0.00025068765)
('cause', 0.0002506876)
('fell', 0.00025068715)
('established', 0.0002506871)
('drumming', 0.00025068692)
('worms', 0.00025068666)
('lives', 0.00025068654)
('walked', 0.00025068645)
('intentions', 0.00025068625)
('marriage', 0.00025068622)
('whose', 0.00025068616)
('ready', 0.0002506857)
('prahova', 0.0002506857)
('depths', 0.00025068535)
('hard', 0.0002506852)
('instead', 0.0002506851)
('tales', 0.00025068497)
('tied', 0.0002506849)
('wild', 0.00025068462)
('killed', 0.00025068436)
('kitchen', 0.0002506842)
('throw', 0.00025068413)
('whatever', 0.000250684)
('pure', 0.00025068357)
('placed', 0.0002506834)
('school', 0.00025068317)
('minutes', 0.00025068302)
('casting', 0.00025068296)
('called', 0.0002506829)
('unnamed', 0.0002506828)
('guitar', 0.00025068264)
('quickly', 0.0002506822)
('fresh', 0.00025068206)
('breast', 0.00025068191)
('nature', 0.0002506818)
('painter', 0.00025068157)

In [126]:

for i in range(1136, 1179):
    print(l1[i])

('generations', 0.00026400774)
('nadine', 0.00026400696)
('continuing', 0.0002640067)
('watching', 0.00026400542)
('names', 0.0002640048)
('translators', 0.00026400472)
('painter', 0.00026400373)
('nature', 0.0002640027)
('breast', 0.00026400236)
('fresh', 0.00026400163)
('quickly', 0.00026400082)
('guitar', 0.00026399863)
('unnamed', 0.00026399785)
('called', 0.00026399703)
('casting', 0.00026399648)
('minutes', 0.00026399633)
('school', 0.00026399546)
('placed', 0.00026399447)
('pure', 0.0002639936)
('whatever', 0.0002639916)
('throw', 0.0002639909)
('kitchen', 0.00026399043)
('killed', 0.0002639896)
('wild', 0.00026398836)
('tied', 0.00026398676)
('tales', 0.0002639862)
('instead', 0.00026398592)
('hard', 0.00026398478)
('depths', 0.00026398446)
('prahova', 0.00026398257)
('ready', 0.00026398257)
('whose', 0.00026397992)
('marriage', 0.0002639799)
('intentions', 0.00026397957)
('walked', 0.00026397835)
('lives', 0.0002639782)
('worms', 0.00026397733)
('drumming', 0.00026397602)
('es

In [128]:

for i in range(3785):
    if l0[i][0]== "poet":
            print(i)

1


In [129]:

for i in range(3785):
    if l1[i][0]== "poet":
            print(i)

1


In [130]:

for i in range(41):
    print(l0[i])

('verse', 0.005966314)
('poet', 0.005565974)
('ovid', 0.0049642525)
('when', 0.004664056)
('other', 0.004160677)
('time', 0.0039610923)
('poem', 0.0035603365)
('work', 0.003258356)
('know', 0.0029588894)
('write', 0.0026574135)
('poets', 0.0025569715)
('naum', 0.0025567992)
('said', 0.002456694)
('poetry', 0.002356873)
('long', 0.0022569722)
('where', 0.0022569187)
('still', 0.0022564116)
('better', 0.0022562025)
('lines', 0.00215656)
('marlowe', 0.0021565487)
('writing', 0.0021560013)
('down', 0.0020562706)
('night', 0.0020560983)
('half', 0.0020560639)
('full', 0.002055618)
('over', 0.0019557113)
('every', 0.0019556393)
('both', 0.0019554493)
('thought', 0.0019554475)
('under', 0.0019552922)
('dryden', 0.0019552046)
('life', 0.0018550719)
('line', 0.001855044)
('words', 0.0018550262)
('between', 0.0018550013)
('sequence', 0.0017553684)
('things', 0.0017553205)
('might', 0.0017553084)
('make', 0.0017551153)
('right', 0.0016551316)
('before', 0.0016550309)


In [131]:

for i in range(41):
    print(l1[i])

('verse', 0.00030230705)
('poet', 0.00029564928)
('other', 0.0002938704)
('ovid', 0.00029200802)
('time', 0.0002874832)
('work', 0.00028705088)
('when', 0.0002867225)
('poem', 0.00028300693)
('naum', 0.00028044012)
('poets', 0.00027954704)
('write', 0.00027933918)
('said', 0.0002788923)
('know', 0.00027794047)
('better', 0.0002772503)
('writing', 0.0002761919)
('still', 0.00027615283)
('dryden', 0.00027614806)
('full', 0.00027609136)
('poetry', 0.0002758482)
('under', 0.00027568787)
('between', 0.0002751058)
('love', 0.00027499296)
('words', 0.00027497308)
('line', 0.00027487817)
('thought', 0.00027487686)
('both', 0.00027487017)
('life', 0.00027473958)
('every', 0.00027387086)
('half', 0.0002737533)
('night', 0.00027358005)
('where', 0.00027349978)
('over', 0.0002734988)
('marlowe', 0.00027333092)
('never', 0.0002733227)
('lines', 0.00027327114)
('long', 0.00027321812)
('only', 0.0002731033)
('water', 0.00027283418)
('down', 0.00027267676)
('each', 0.00027265097)
('something', 0.00027

In [132]:

for i in range(3785):
    if l0[i][0]== "freedom":
            print(i)

876


In [133]:

for i in range(3785):
    if l1[i][0]== "freedom":
            print(i)

1118


In [135]:

for i in range(855, 899):
    print(l0[i])

('watching', 0.0002506813)
('continuing', 0.000250681)
('nadine', 0.00025068095)
('generations', 0.00025068087)
('departures', 0.0002506808)
('panthers', 0.0002506808)
('spectacular', 0.00025068075)
('magic', 0.00025068066)
('ditches', 0.00025068066)
('everyday', 0.00025068055)
('2000', 0.0002506803)
('beaten', 0.0002506799)
('sleeping', 0.0002506794)
('forms', 0.00025067938)
('accomplishes', 0.00025067915)
('hiding', 0.00025067874)
('worlds', 0.00025067868)
('favour', 0.0002506786)
('stairway', 0.0002506786)
('tables', 0.00025067833)
('lions', 0.00025067833)
('freedom', 0.0002506781)
('forehead', 0.00025067717)
('prognostications', 0.00025067703)
('nails', 0.00025067694)
('species', 0.00025067694)
('amidst', 0.0002506766)
('center', 0.00025067647)
('xxiii', 0.00025067644)
('values', 0.00025067627)
('usual', 0.00025067618)
('desperate', 0.00025067595)
('ambushed', 0.00025067592)
('indefinite', 0.0002506759)
('sides', 0.0002506759)
('roared', 0.00025067563)
('xxvii', 0.00025067522)
('fa

In [134]:

for i in range(1097, 1139):
    print(l1[i])

('dreams', 0.00026404308)
('bored', 0.0002640417)
('history', 0.00026404124)
('unclear', 0.00026404063)
('happen', 0.0002640389)
('xxvii', 0.00026403752)
('farther', 0.0002640373)
('roared', 0.0002640352)
('sides', 0.00026403385)
('indefinite', 0.00026403356)
('ambushed', 0.00026403347)
('desperate', 0.00026403295)
('usual', 0.00026403222)
('values', 0.00026403167)
('xxiii', 0.00026403056)
('center', 0.00026403047)
('amidst', 0.00026402975)
('species', 0.0002640283)
('nails', 0.00026402794)
('prognostications', 0.00026402756)
('forehead', 0.00026402713)
('freedom', 0.00026402227)
('tables', 0.00026402102)
('lions', 0.00026402087)
('favour', 0.00026401968)
('stairway', 0.00026401965)
('worlds', 0.00026401918)
('hiding', 0.00026401886)
('accomplishes', 0.00026401645)
('forms', 0.0002640154)
('sleeping', 0.0002640153)
('beaten', 0.00026401272)
('2000', 0.00026401054)
('everyday', 0.00026400937)
('magic', 0.00026400865)
('ditches', 0.0002640086)
('spectacular', 0.00026400856)
('panthers', 

In [139]:

for i in range(3785):
    if l0[i][0]== "rhythms":
            print(i)

965


In [138]:

for i in range(3785):
    if l1[i][0]== "rhythms":
            print(i)

1028


In [140]:

for i in range(944, 985):
    print(l0[i])

('xxvi', 0.00025066728)
('taken', 0.00025066713)
('grenades', 0.0002506671)
('rhymes', 0.000250667)
('dawn', 0.00025066693)
('whether', 0.00025066684)
('nicely', 0.0002506666)
('lime', 0.0002506665)
('sort', 0.00025066626)
('church', 0.00025066626)
('culture', 0.0002506662)
('climax', 0.00025066614)
('poetic', 0.00025066605)
('performed', 0.00025066594)
('pipes', 0.00025066594)
('riverbank', 0.00025066585)
('obscure', 0.0002506658)
('dies', 0.0002506656)
('suspiciously', 0.0002506656)
('hopeless', 0.00025066538)
('fall', 0.00025066506)
('rhythms', 0.00025066506)
('processes', 0.00025066486)
('sphinx', 0.00025066483)
('lover', 0.00025066413)
('chosen', 0.00025066402)
('occasionally', 0.00025066384)
('seem', 0.00025066384)
('lines—', 0.00025066378)
('capacities', 0.00025066367)
('collected', 0.0002506631)
('characters', 0.00025066297)
('pair', 0.00025066282)
('doubt', 0.0002506628)
('draft', 0.00025066268)
('afternoon', 0.00025066227)
('speech', 0.00025066227)
('buffalo', 0.0002506622)
(

In [141]:

for i in range(1007, 1048):
    print(l1[i])

('vegetal', 0.00026410638)
('anticus', 0.00026410513)
('receipt', 0.00026410513)
('analogy', 0.00026410504)
('afternoon', 0.00026410486)
('buffalo', 0.00026410448)
('speech', 0.00026410443)
('draft', 0.00026410256)
('doubt', 0.00026410213)
('pair', 0.00026410155)
('characters', 0.00026410123)
('collected', 0.00026410038)
('capacities', 0.00026409738)
('occasionally', 0.0002640967)
('lines—', 0.0002640966)
('seem', 0.00026409645)
('chosen', 0.00026409575)
('lover', 0.00026409485)
('sphinx', 0.00026409136)
('processes', 0.00026409057)
('fall', 0.00026409025)
('rhythms', 0.0002640899)
('hopeless', 0.00026408827)
('dies', 0.0002640874)
('suspiciously', 0.0002640874)
('obscure', 0.00026408618)
('riverbank', 0.0002640858)
('performed', 0.00026408548)
('pipes', 0.0002640854)
('poetic', 0.0002640842)
('climax', 0.00026408417)
('culture', 0.00026408408)
('church', 0.0002640838)
('sort', 0.00026408368)
('lime', 0.00026408286)
('nicely', 0.0002640822)
('whether', 0.000264081)
('scarf', 0.00026408

In [142]:

for i in range(3785):
    if l0[i][0]== "time":
            print(i)

5


In [143]:

for i in range(3785):
    if l1[i][0]== "time":
            print(i)

4


In [144]:

for i in range(42):
    print(l0[i])

('verse', 0.005966314)
('poet', 0.005565974)
('ovid', 0.0049642525)
('when', 0.004664056)
('other', 0.004160677)
('time', 0.0039610923)
('poem', 0.0035603365)
('work', 0.003258356)
('know', 0.0029588894)
('write', 0.0026574135)
('poets', 0.0025569715)
('naum', 0.0025567992)
('said', 0.002456694)
('poetry', 0.002356873)
('long', 0.0022569722)
('where', 0.0022569187)
('still', 0.0022564116)
('better', 0.0022562025)
('lines', 0.00215656)
('marlowe', 0.0021565487)
('writing', 0.0021560013)
('down', 0.0020562706)
('night', 0.0020560983)
('half', 0.0020560639)
('full', 0.002055618)
('over', 0.0019557113)
('every', 0.0019556393)
('both', 0.0019554493)
('thought', 0.0019554475)
('under', 0.0019552922)
('dryden', 0.0019552046)
('life', 0.0018550719)
('line', 0.001855044)
('words', 0.0018550262)
('between', 0.0018550013)
('sequence', 0.0017553684)
('things', 0.0017553205)
('might', 0.0017553084)
('make', 0.0017551153)
('right', 0.0016551316)
('before', 0.0016550309)
('here', 0.0016548585)


In [145]:

for i in range(3785):
    if l0[i][0]== "artist":
            print(i)

230


In [146]:

for i in range(3785):
    if l1[i][0]== "artist":
            print(i)

154


In [147]:

for i in range(209, 250):
    print(l0[i])

('order', 0.0006518219)
('stones', 0.0006518189)
('took', 0.0006518176)
('matter', 0.0006518133)
('jacket', 0.0006518091)
('conference', 0.0006518006)
('space', 0.000651799)
('soon', 0.0006517946)
('stuck', 0.0006517891)
('drunk', 0.000651785)
('goes', 0.0006517648)
('modern', 0.00065176334)
('latin', 0.00065176067)
('start', 0.0006517539)
('knew', 0.0006517534)
('beyond', 0.0006517504)
('surrealist', 0.0006517496)
('sleep', 0.00065174507)
('sense', 0.0006517411)
('written', 0.00065171137)
('taste', 0.0006516993)
('artist', 0.0006516839)
('wind', 0.0006516342)
('comes', 0.0006516277)
('used', 0.0006516219)
('rest', 0.0006516077)
('tower', 0.0005517007)
('worst', 0.0005516782)
('lips', 0.0005516778)
('asked', 0.0005516723)
('memories', 0.0005516694)
('waiting', 0.00055166177)
('zenobia', 0.0005516551)
('poems', 0.00055163365)
('language', 0.00055162713)
('bird', 0.0005516147)
('case', 0.00055161444)
('illustrations', 0.00055161404)
('rock', 0.00055161363)
('stop', 0.0005516103)
('poor',

In [148]:

for i in range(133, 174):
    print(l1[i])

('personal', 0.00026755294)
('voice', 0.0002675379)
('amores', 0.00026752398)
('window', 0.00026750585)
('used', 0.0002675028)
('comes', 0.00026747395)
('same', 0.00026744977)
('beat', 0.0002674458)
('near', 0.00026744095)
('wind', 0.00026743958)
('glass', 0.0002674376)
('least', 0.0002674228)
('problem', 0.000267412)
('always', 0.0002674084)
('grass', 0.0002674067)
('walk', 0.00026737724)
('rhythm', 0.00026732695)
('sounds', 0.00026724415)
('think', 0.00026721682)
('less', 0.00026718242)
('making', 0.0002671822)
('artist', 0.00026718085)
('tell', 0.00026717567)
('path', 0.00026713414)
('taste', 0.00026709898)
('wrote', 0.0002670834)
('hair', 0.00026708186)
('memory', 0.00026705718)
('wanted', 0.00026705532)
('nothing', 0.00026704787)
('truth', 0.00026704583)
('written', 0.00026703725)
('egos', 0.0002670012)
('hear', 0.0002669788)
('talking', 0.00026693352)
('door', 0.00026691132)
('fourfour', 0.00026688687)
('sense', 0.00026688218)
('metamorphoses', 0.00026687546)
('sleep', 0.00026686

In [149]:

for i in range(3785):
    if l0[i][0]== "rhymes":
            print(i)

947


In [150]:

for i in range(3785):
    if l1[i][0]== "rhymes":
            print(i)

1046


In [151]:

for i in range(926, 967):
    print(l0[i])

('andré', 0.00025066966)
('sentiments', 0.00025066952)
('store', 0.00025066952)
('trip', 0.0002506693)
('played', 0.0002506693)
('calea', 0.0002506688)
('moonlight', 0.00025066876)
('morning', 0.00025066873)
('sing', 0.00025066864)
('thrumming', 0.00025066864)
('facing', 0.0002506684)
('follow', 0.00025066835)
('seen', 0.0002506683)
('list', 0.00025066797)
('swimming', 0.00025066792)
('type', 0.00025066777)
('murderous', 0.00025066774)
('glistening', 0.00025066768)
('xxvi', 0.00025066728)
('taken', 0.00025066713)
('grenades', 0.0002506671)
('rhymes', 0.000250667)
('dawn', 0.00025066693)
('whether', 0.00025066684)
('nicely', 0.0002506666)
('lime', 0.0002506665)
('sort', 0.00025066626)
('church', 0.00025066626)
('culture', 0.0002506662)
('climax', 0.00025066614)
('poetic', 0.00025066605)
('performed', 0.00025066594)
('pipes', 0.00025066594)
('riverbank', 0.00025066585)
('obscure', 0.0002506658)
('dies', 0.0002506656)
('suspiciously', 0.0002506656)
('hopeless', 0.00025066538)
('fall', 0.0

In [152]:

for i in range(1025, 1066):
    print(l1[i])

('sphinx', 0.00026409136)
('processes', 0.00026409057)
('fall', 0.00026409025)
('rhythms', 0.0002640899)
('hopeless', 0.00026408827)
('dies', 0.0002640874)
('suspiciously', 0.0002640874)
('obscure', 0.00026408618)
('riverbank', 0.0002640858)
('performed', 0.00026408548)
('pipes', 0.0002640854)
('poetic', 0.0002640842)
('climax', 0.00026408417)
('culture', 0.00026408408)
('church', 0.0002640838)
('sort', 0.00026408368)
('lime', 0.00026408286)
('nicely', 0.0002640822)
('whether', 0.000264081)
('scarf', 0.0002640809)
('dawn', 0.0002640807)
('rhymes', 0.00026408018)
('grenades', 0.00026407937)
('taken', 0.0002640793)
('xxvi', 0.00026407873)
('glistening', 0.00026407622)
('murderous', 0.00026407585)
('type', 0.0002640758)
('swimming', 0.00026407524)
('list', 0.00026407486)
('facing', 0.000264073)
('seen', 0.00026407288)
('follow', 0.00026407262)
('sing', 0.0002640715)
('thrumming', 0.0002640714)
('moonlight', 0.00026407102)
('calea', 0.00026407096)
('morning', 0.0002640709)
('trip', 0.00026

In [153]:

for i in range(3785):
    if l0[i][0]== "rebirth":
            print(i)

In [154]:

for i in range(3785):
    if l1[i][0]== "rebirth":
            print(i)

In [155]:

for i in range(3785):
    if l0[i][0]== "renaissance":
            print(i)

692


In [156]:

for i in range(3785):
    if l1[i][0]== "renaissance":
            print(i)

400


In [157]:

for i in range(671, 712):
    print(l0[i])


('errors', 0.0003508811)
('prove', 0.00035088102)
('hears', 0.00035088096)
('shoes', 0.00035088067)
('question', 0.0003508804)
('giant', 0.00035088015)
('briefcase', 0.0003508788)
('believes', 0.00035087837)
('gone', 0.00035087622)
('turns', 0.00035087555)
('agony', 0.00035087543)
('writers', 0.00035087508)
('denver', 0.00035087462)
('knows', 0.00035087197)
('major', 0.0003508678)
('symbol', 0.00035086388)
('british', 0.0003508634)
('bottle', 0.00035086164)
('moment', 0.00035085852)
('diva', 0.0003508572)
('smelled', 0.00035085395)
('renaissance', 0.00035085384)
('willows', 0.0003508468)
('works', 0.00035084572)
('enjoying', 0.0003508182)
('lygia', 0.00035081457)
('descrierea', 0.00035081405)
('takes', 0.00035079807)
('american', 0.0002507637)
('incredible', 0.00025074606)
('wheels', 0.00025074455)
('bearer', 0.0002507409)
('held', 0.00025074056)
('favourite', 0.0002507385)
('rifle', 0.00025073625)
('deeply', 0.00025073488)
('food', 0.00025073462)
('fast', 0.00025073337)
('blanket', 0.

In [158]:
for i in range(379, 420):
    print(l1[i])

('horse', 0.0002653207)
('agree', 0.0002653128)
('because', 0.00026530382)
('house', 0.00026529137)
('thinking', 0.00026529134)
('study', 0.0002652895)
('drinks', 0.000265284)
('bank', 0.00026528214)
('literature', 0.00026527428)
('wrong', 0.00026527393)
('greater', 0.00026526864)
('bought', 0.00026526823)
('various', 0.00026526602)
('gets', 0.00026525793)
('breton', 0.00026525478)
('nobody', 0.00026525033)
('works', 0.0002652469)
('famous', 0.000265245)
('willows', 0.00026524105)
('cold', 0.00026522676)
('zenobia', 0.00026522623)
('renaissance', 0.00026520464)
('smelled', 0.00026520347)
('waiting', 0.0002651913)
('diva', 0.00026518689)
('unexpected', 0.00026518124)
('moment', 0.0002651793)
('year', 0.00026517652)
('bottle', 0.00026516357)
('rise', 0.0002651626)
('drummer', 0.0002651557)
('british', 0.00026515452)
('symbol', 0.00026515205)
('memories', 0.00026515164)
('flames', 0.00026514588)
('middle', 0.00026514186)
('asked', 0.00026513543)
('major', 0.00026513173)
('turned', 0.00026

In [159]:

for i in range(3785):
    if l0[i][0]== "differences":
            print(i)

1052


In [163]:

for i in range(3785):
    if l1[i][0]== "differences":
            print(i)

939


In [161]:

for i in range(1031, 1072):
    print(l0[i])


('customs', 0.00025065633)
('curiouser', 0.0002506563)
('flowers', 0.00025065607)
('importance', 0.00025065607)
('advise', 0.00025065578)
('shakespeare', 0.00025065566)
('coat', 0.00025065534)
('echoes', 0.00025065526)
('deptford', 0.00025065485)
('young', 0.00025065476)
('friendship', 0.0002506545)
('teodorescu', 0.00025065444)
('ears', 0.00025065444)
('eaten', 0.00025065392)
('hour', 0.00025065357)
('elizabethan', 0.0002506535)
('genuine', 0.00025065333)
('paradise', 0.00025065313)
('tragic', 0.00025065299)
('tomato', 0.00025065278)
('tiny', 0.00025065267)
('differences', 0.00025065258)
('damned', 0.00025065243)
('vigorous', 0.00025065243)
('oneofakind', 0.0002506523)
('curious', 0.00025065217)
('islands', 0.00025065202)
('speaking', 0.00025065185)
('bluegreen', 0.0002506518)
('knowledge', 0.0002506515)
('watch', 0.00025065133)
('looks', 0.00025065112)
('grand', 0.00025065112)
('urbanrural', 0.00025065106)
('village', 0.00025065104)
('remembered', 0.0002506506)
('wrapped', 0.00025065

In [164]:

for i in range(918, 959):
    print(l1[i])

('idol', 0.00026416857)
('western', 0.00026416793)
('complex', 0.00026416776)
('lunar', 0.0002641669)
('mothers', 0.00026416633)
('wrapped', 0.0002641658)
('remembered', 0.00026416566)
('urbanrural', 0.00026416325)
('village', 0.00026416325)
('grand', 0.00026416266)
('looks', 0.00026416246)
('watch', 0.0002641616)
('knowledge', 0.0002641607)
('bluegreen', 0.00026415888)
('speaking', 0.00026415862)
('islands', 0.0002641579)
('curious', 0.0002641574)
('plot', 0.0002641574)
('oneofakind', 0.00026415667)
('damned', 0.0002641562)
('vigorous', 0.00026415614)
('differences', 0.0002641549)
('tiny', 0.00026415475)
('tomato', 0.00026415393)
('tragic', 0.000264153)
('paradise', 0.000264152)
('genuine', 0.00026415128)
('elizabethan', 0.00026415044)
('hour', 0.0002641502)
('eaten', 0.00026414826)
('teodorescu', 0.0002641454)
('ears', 0.0002641452)
('friendship', 0.00026414497)
('young', 0.0002641437)
('deptford', 0.00026414328)
('echoes', 0.00026414148)
('coat', 0.00026414066)
('shakespeare', 0.000

In [165]:

for i in range(3785):
    if l0[i][0]== "meaning":
            print(i)

275


In [166]:

for i in range(3785):
    if l1[i][0]== "meaning":
            print(i)

283


In [167]:

for i in range(254, 295):
    print(l0[i])


('tangled', 0.0005515951)
('enough', 0.0005515893)
('foot', 0.0005515845)
('discussion', 0.0005515798)
('project', 0.0005515763)
('derby', 0.00055157003)
('next', 0.0005515693)
('fact', 0.0005515688)
('rhyme', 0.00055156834)
('past', 0.0005515683)
('albastru', 0.0005515652)
('become', 0.0005515641)
('alchemical', 0.0005515623)
('exiled', 0.0005515599)
('sound', 0.00055155763)
('marvellous', 0.0005515529)
('eutychia', 0.00055155216)
('malul', 0.00055155176)
('mist', 0.000551547)
('artists', 0.00055154663)
('loved', 0.0005515449)
('meaning', 0.0005515447)
('toward', 0.00055154454)
('stone', 0.00055154314)
('girl', 0.00055153883)
('constanta', 0.00055153784)
('blood', 0.00055153354)
('based', 0.00055152853)
('against', 0.0005515245)
('known', 0.00055152446)
('looked', 0.00055152044)
('romantic', 0.00055151456)
('distance', 0.00055151386)
('dream', 0.00055150955)
('green', 0.0005515067)
('london', 0.00055150496)
('theme', 0.0005515028)
('cealaltă', 0.0005515024)
('published', 0.00055149966

In [168]:
for i in range(262, 303):
    print(l1[i])

('distance', 0.00026596437)
('romantic', 0.00026596076)
('finding', 0.0002659461)
('looked', 0.00026593023)
('english', 0.0002659175)
('against', 0.00026590808)
('known', 0.00026590808)
('becomes', 0.00026588884)
('based', 0.00026588768)
('breasts', 0.0002658864)
('gallery', 0.00026588168)
('bucharest', 0.00026587278)
('blood', 0.00026586137)
('till', 0.00026585895)
('state', 0.00026584222)
('constanta', 0.0002658393)
('style', 0.00026583666)
('girl', 0.00026583372)
('stone', 0.0002658113)
('toward', 0.00026580394)
('loved', 0.00026580266)
('meaning', 0.00026580234)
('artists', 0.00026579318)
('evening', 0.00026579187)
('mist', 0.00026579064)
('part', 0.00026578474)
('waters', 0.0002657821)
('brauner', 0.000265776)
('malul', 0.00026576564)
('eutychia', 0.0002657644)
('marvellous', 0.00026575974)
('brought', 0.000265744)
('sound', 0.0002657362)
('terrible', 0.00026572982)
('exiled', 0.00026572254)
('alchemical', 0.00026571116)
('become', 0.0002657005)
('albastru', 0.0002656963)
('stand'

In [169]:

for i in range(3785):
    if l0[i][0] == "unexpected":
            print(i)

365


In [170]:

for i in range(3785):
    if l1[i][0] == "unexpected":
            print(i)

404


In [171]:

for i in range(344, 385):
    print(l0[i])

('berry', 0.00045129718)
('concept', 0.00045129078)
('couldn', 0.00045129072)
('sent', 0.00045129028)
('stopped', 0.0004512855)
('seems', 0.0004512853)
('wake', 0.00045128417)
('touch', 0.00045128388)
('ritual', 0.0004512832)
('clear', 0.00045128312)
('front', 0.00045128085)
('feeling', 0.000451278)
('almost', 0.00045127593)
('subjects', 0.0004512744)
('tortured', 0.00045127343)
('turned', 0.00045127288)
('middle', 0.00045126842)
('flames', 0.0004512679)
('drummer', 0.00045126586)
('rise', 0.00045126452)
('year', 0.00045126182)
('unexpected', 0.000451261)
('cold', 0.00045125233)
('famous', 0.00045124878)
('nobody', 0.00045124773)
('breton', 0.00045124686)
('gets', 0.0004512463)
('various', 0.0004512447)
('bought', 0.00045124433)
('greater', 0.00045124415)
('wrong', 0.00045124328)
('literature', 0.0004512432)
('bank', 0.00045124156)
('drinks', 0.00045124127)
('study', 0.00045124022)
('house', 0.00045124002)
('thinking', 0.00045123984)
('because', 0.00045123755)
('agree', 0.00045123565)


In [172]:

for i in range(383, 424):
    print(l1[i])

('thinking', 0.00026529134)
('study', 0.0002652895)
('drinks', 0.000265284)
('bank', 0.00026528214)
('literature', 0.00026527428)
('wrong', 0.00026527393)
('greater', 0.00026526864)
('bought', 0.00026526823)
('various', 0.00026526602)
('gets', 0.00026525793)
('breton', 0.00026525478)
('nobody', 0.00026525033)
('works', 0.0002652469)
('famous', 0.000265245)
('willows', 0.00026524105)
('cold', 0.00026522676)
('zenobia', 0.00026522623)
('renaissance', 0.00026520464)
('smelled', 0.00026520347)
('waiting', 0.0002651913)
('diva', 0.00026518689)
('unexpected', 0.00026518124)
('moment', 0.0002651793)
('year', 0.00026517652)
('bottle', 0.00026516357)
('rise', 0.0002651626)
('drummer', 0.0002651557)
('british', 0.00026515452)
('symbol', 0.00026515205)
('memories', 0.00026515164)
('flames', 0.00026514588)
('middle', 0.00026514186)
('asked', 0.00026513543)
('major', 0.00026513173)
('turned', 0.00026511902)
('tortured', 0.0002651162)
('subjects', 0.00026511142)
('knows', 0.00026510964)
('lips', 0.0

In [173]:

for i in range(3785):
    if l0[i][0]== "plan":
            print(i)

78


In [174]:

for i in range(3785):
    if l1[i][0]== "plan":
            print(i)

81


In [175]:

for i in range(57, 99):
    print(l0[i])

('again', 0.0014540444)
('look', 0.0014539777)
('each', 0.0014538596)
('others', 0.0013540927)
('away', 0.0013539506)
('eyes', 0.0013538802)
('most', 0.0013538484)
('found', 0.0013536845)
('came', 0.0013536307)
('take', 0.0013535818)
('only', 0.0013533706)
('went', 0.0012537509)
('head', 0.0012537395)
('world', 0.0012537257)
('perhaps', 0.0012535215)
('ancient', 0.0012534529)
('alter', 0.0012533682)
('three', 0.0012531913)
('above', 0.001153581)
('death', 0.0011534953)
('face', 0.0011533841)
('plan', 0.0011532906)
('wine', 0.0011532522)
('later', 0.0011531275)
('blue', 0.0011530515)
('says', 0.001153038)
('side', 0.0011528659)
('translation', 0.0010532183)
('until', 0.0010532007)
('started', 0.001053197)
('black', 0.0010530917)
('find', 0.0010530588)
('light', 0.0010530384)
('moon', 0.0010530237)
('second', 0.0010530044)
('come', 0.0010529625)
('everything', 0.0010529559)
('place', 0.0010529556)
('verses', 0.0010528893)
('earth', 0.0010528528)
('form', 0.0010528367)
('silence', 0.00105

In [176]:

for i in range(60, 101):
    print(l1[i])

('before', 0.0002707399)
('says', 0.00027062916)
('most', 0.00027060544)
('ancient', 0.00027056565)
('blue', 0.0002705586)
('eyes', 0.00027043445)
('silence', 0.00027041862)
('made', 0.00027034653)
('romanian', 0.00027030715)
('right', 0.00027020922)
('perhaps', 0.0002702087)
('later', 0.0002701604)
('away', 0.00027006742)
('course', 0.00026999714)
('good', 0.00026959524)
('form', 0.000269577)
('athanor', 0.00026954978)
('wine', 0.0002695088)
('earth', 0.00026948933)
('coming', 0.0002694289)
('others', 0.00026932464)
('plan', 0.00026930892)
('verses', 0.00026930167)
('read', 0.00026917702)
('translations', 0.00026917487)
('left', 0.00026916078)
('world', 0.0002691376)
('tree', 0.0002690837)
('head', 0.00026906483)
('best', 0.00026905094)
('went', 0.00026900566)
('grow', 0.00026897786)
('five', 0.000268956)
('place', 0.00026895374)
('everything', 0.00026895298)
('come', 0.0002689184)
('face', 0.00026881965)
('need', 0.0002687301)
('second', 0.00026870024)
('moon', 0.0002685978)
('years'

In [177]:

for i in range(3785):
    if l0[i][0]== "mind":
            print(i)

316


In [178]:

for i in range(3785):
    if l1[i][0]== "mind":
            print(i)

175


In [179]:

for i in range(295, 336):
    print(l0[i])

('anyway', 0.0005514926)
('despair', 0.0005514893)
('telling', 0.00055148697)
('pass', 0.0005514781)
('fața', 0.00055147696)
('master', 0.00055147626)
('myself', 0.0005514739)
('rental', 0.0005514735)
('lance', 0.00055146875)
('vintage', 0.0005514583)
('thus', 0.00055145036)
('council', 0.0005514432)
('especially', 0.00055144203)
('girls', 0.000551435)
('happy', 0.00055143284)
('partea', 0.0005514313)
('finger', 0.00055142667)
('passed', 0.0005514195)
('ashes', 0.00055140705)
('painting', 0.0005513837)
('little', 0.0005513822)
('mind', 0.0005513486)
('metamorphoses', 0.0005513394)
('fourfour', 0.000551337)
('talking', 0.0005513283)
('gestures', 0.00045142855)
('book', 0.0004513758)
('towards', 0.00045137198)
('grown', 0.000451355)
('drink', 0.00045135475)
('understanding', 0.00045135446)
('fingers', 0.00045135262)
('desperately', 0.00045135024)
('lack', 0.00045134325)
('worthy', 0.00045134043)
('spring', 0.00045133993)
('airport', 0.0004513395)
('debate', 0.00045133554)
('anyone', 0.00

In [180]:

for i in range(154, 195):
    print(l1[i])

('artist', 0.00026718085)
('tell', 0.00026717567)
('path', 0.00026713414)
('taste', 0.00026709898)
('wrote', 0.0002670834)
('hair', 0.00026708186)
('memory', 0.00026705718)
('wanted', 0.00026705532)
('nothing', 0.00026704787)
('truth', 0.00026704583)
('written', 0.00026703725)
('egos', 0.0002670012)
('hear', 0.0002669788)
('talking', 0.00026693352)
('door', 0.00026691132)
('fourfour', 0.00026688687)
('sense', 0.00026688218)
('metamorphoses', 0.00026687546)
('sleep', 0.00026686073)
('surrealist', 0.00026683742)
('beyond', 0.00026683303)
('mind', 0.0002668281)
('knew', 0.0002668166)
('start', 0.0002668149)
('alone', 0.0002668)
('four', 0.00026679234)
('latin', 0.00026677913)
('presentation', 0.00026677013)
('modern', 0.00026676457)
('goes', 0.00026675852)
('small', 0.00026672176)
('little', 0.00026665148)
('drunk', 0.00026665142)
('painting', 0.00026664493)
('stuck', 0.00026663105)
('home', 0.000266614)
('woman', 0.00026660404)
('soon', 0.0002666021)
('space', 0.00026657927)
('conference

In [181]:

for i in range(3785):
    if l0[i][0]== "wine":
            print(i)

79


In [182]:

for i in range(3785):
    if l1[i][0]== "wine":
            print(i)

77


In [184]:

for i in range(57, 100):
    print(l0[i])


('again', 0.0014540444)
('look', 0.0014539777)
('each', 0.0014538596)
('others', 0.0013540927)
('away', 0.0013539506)
('eyes', 0.0013538802)
('most', 0.0013538484)
('found', 0.0013536845)
('came', 0.0013536307)
('take', 0.0013535818)
('only', 0.0013533706)
('went', 0.0012537509)
('head', 0.0012537395)
('world', 0.0012537257)
('perhaps', 0.0012535215)
('ancient', 0.0012534529)
('alter', 0.0012533682)
('three', 0.0012531913)
('above', 0.001153581)
('death', 0.0011534953)
('face', 0.0011533841)
('plan', 0.0011532906)
('wine', 0.0011532522)
('later', 0.0011531275)
('blue', 0.0011530515)
('says', 0.001153038)
('side', 0.0011528659)
('translation', 0.0010532183)
('until', 0.0010532007)
('started', 0.001053197)
('black', 0.0010530917)
('find', 0.0010530588)
('light', 0.0010530384)
('moon', 0.0010530237)
('second', 0.0010530044)
('come', 0.0010529625)
('everything', 0.0010529559)
('place', 0.0010529556)
('verses', 0.0010528893)
('earth', 0.0010528528)
('form', 0.0010528367)
('silence', 0.00105

In [183]:

for i in range(56, 99):
    print(l1[i])

('after', 0.00027112634)
('sequence', 0.00027107768)
('alter', 0.00027100928)
('word', 0.00027074935)
('before', 0.0002707399)
('says', 0.00027062916)
('most', 0.00027060544)
('ancient', 0.00027056565)
('blue', 0.0002705586)
('eyes', 0.00027043445)
('silence', 0.00027041862)
('made', 0.00027034653)
('romanian', 0.00027030715)
('right', 0.00027020922)
('perhaps', 0.0002702087)
('later', 0.0002701604)
('away', 0.00027006742)
('course', 0.00026999714)
('good', 0.00026959524)
('form', 0.000269577)
('athanor', 0.00026954978)
('wine', 0.0002695088)
('earth', 0.00026948933)
('coming', 0.0002694289)
('others', 0.00026932464)
('plan', 0.00026930892)
('verses', 0.00026930167)
('read', 0.00026917702)
('translations', 0.00026917487)
('left', 0.00026916078)
('world', 0.0002691376)
('tree', 0.0002690837)
('head', 0.00026906483)
('best', 0.00026905094)
('went', 0.00026900566)
('grow', 0.00026897786)
('five', 0.000268956)
('place', 0.00026895374)
('everything', 0.00026895298)
('come', 0.0002689184)
('

In [185]:

for i in range(3785):
    if l0[i][0]== "ideas":
            print(i)

544


In [186]:

for i in range(3785):
    if l1[i][0]== "ideas":
            print(i)

597


In [187]:

for i in range(523, 564):
    print(l0[i])

('anymore', 0.00035097246)
('easy', 0.00035097223)
('colored', 0.0003509719)
('2001', 0.00035097075)
('millions', 0.00035096987)
('followed', 0.000350969)
('tide', 0.0003509685)
('boat', 0.00035096725)
('movement', 0.00035096693)
('changed', 0.00035096658)
('animaltree', 0.00035096504)
('twelve', 0.00035096487)
('unless', 0.0003509648)
('traditional', 0.0003509646)
('starts', 0.0003509634)
('lucrurile', 0.0003509628)
('dropped', 0.0003509627)
('fixed', 0.00035096242)
('grew', 0.00035096236)
('prose', 0.00035096187)
('themselves', 0.00035096184)
('ideas', 0.00035096117)
('going', 0.00035096015)
('grant', 0.00035095928)
('instance', 0.00035095916)
('pale', 0.00035095864)
('ludicrous', 0.00035095864)
('hovering', 0.0003509586)
('pattern', 0.00035095814)
('yourself', 0.000350958)
('shape', 0.00035095745)
('prologue', 0.00035095718)
('hope', 0.0003509562)
('speak', 0.0003509551)
('emptied', 0.00035095497)
('lead', 0.00035095486)
('mouth', 0.0003509541)
('regime', 0.00035095372)
('waits', 0.

In [188]:

for i in range(576, 617):
    print(l1[i])

('leave', 0.0002646905)
('grown', 0.00026469034)
('paris', 0.0002646901)
('books', 0.00026468895)
('waits', 0.00026468406)
('regime', 0.00026468304)
('mouth', 0.00026468138)
('lead', 0.00026467702)
('emptied', 0.00026467687)
('speak', 0.0002646765)
('hope', 0.00026467023)
('prologue', 0.0002646651)
('shape', 0.00026466412)
('yourself', 0.00026466078)
('pattern', 0.00026466014)
('ludicrous', 0.00026465754)
('hovering', 0.00026465743)
('pale', 0.00026465702)
('instance', 0.0002646545)
('grant', 0.00026465382)
('going', 0.00026464922)
('ideas', 0.00026464346)
('themselves', 0.00026464072)
('prose', 0.00026464066)
('grew', 0.00026463845)
('fixed', 0.0002646376)
('dropped', 0.00026463633)
('lucrurile', 0.00026463607)
('starts', 0.00026463263)
('traditional', 0.00026462635)
('unless', 0.0002646249)
('twelve', 0.0002646245)
('animaltree', 0.00026462405)
('changed', 0.00026461642)
('movement', 0.0002646142)
('boat', 0.00026461226)
('tide', 0.00026460586)
('followed', 0.00026460367)
('involved'

In [189]:

for i in range(3785):
    if l0[i][0]== "stopped":
            print(i)

348


In [193]:

for i in range(3785):
    if l0[i][0]== "patient":
            print(i)

1874


In [194]:
for i in range(3785):
    if l0[i][0] == "caring":
        print(i)

1940


In [195]:

for i in range(3785):
    if l0[i][0] == "complements":
        print(i)

1430


In [196]:

for i in range(3785):
    if l0[i][0] == "sober":
        print(i)

1655


In [197]:

for i in range(3785):
    if l0[i][0] == "ramblings":
        print(i)

2422


In [202]:

for i in range(3785):
    if l1[i][0] == "ramblings":
        print(i)

2657


In [198]:

for i in range(3785):
    if l0[i][0] == "swept":
        print(i)

1264


In [199]:

for i in range(3785):
    if l0[i][0] == "brushed":
        print(i)

2509


In [204]:

for i in range(3785):
    if l1[i][0] == "brushed":
        print(i)

2571


In [200]:

for i in range(3785):
    if l0[i][0] == "problem":
        print(i)

128


In [201]:

for i in range(2400, 2443):
    print(l0[i])

('apart', 0.00015037559)
('dabbler', 0.00015037556)
('elizabethanrestoration', 0.00015037555)
('heap', 0.00015037555)
('transparent—surely', 0.00015037545)
('position', 0.00015037543)
('2020', 0.0001503754)
('one', 0.00015037539)
('fragmented', 0.00015037536)
('1948', 0.00015037533)
('jousting', 0.00015037533)
('eyed', 0.00015037533)
('collages', 0.00015037532)
('fighting', 0.0001503753)
('him…', 0.00015037529)
('terrestrialization', 0.00015037526)
('brags', 0.00015037526)
('resonates', 0.00015037526)
('francis', 0.00015037523)
('groundbreaking', 0.00015037521)
('schwitters', 0.00015037518)
('date', 0.00015037518)
('ramblings', 0.00015037518)
('xxxvii', 0.00015037516)
('primary', 0.00015037514)
('career', 0.00015037514)
('nail', 0.00015037511)
('xxxiii', 0.0001503751)
('build', 0.0001503751)
('foxwitches', 0.0001503751)
('singing', 0.00015037508)
('swotting', 0.00015037508)
('shells', 0.00015037508)
('envisage', 0.00015037508)
('reclaiming', 0.00015037504)
('siblings', 0.00015037502)
(

In [203]:

for i in range(2636, 2678):
    print(l1[i])

('reminiscent', 0.00026351822)
('persistently', 0.0002635182)
('mumble', 0.0002635178)
('lurking', 0.0002635177)
('bigger', 0.0002635176)
('lacquer', 0.00026351752)
('catch', 0.00026351746)
('block', 0.00026351743)
('wondering', 0.00026351726)
('siblings', 0.00026351717)
('envisage', 0.000263517)
('reclaiming', 0.000263517)
('singing', 0.00026351694)
('shells', 0.00026351685)
('foxwitches', 0.00026351682)
('swotting', 0.0002635168)
('nail', 0.00026351676)
('build', 0.00026351676)
('xxxiii', 0.00026351667)
('career', 0.00026351656)
('xxxvii', 0.00026351656)
('ramblings', 0.0002635165)
('groundbreaking', 0.00026351638)
('primary', 0.00026351635)
('date', 0.0002635163)
('schwitters', 0.00026351618)
('terrestrialization', 0.00026351606)
('francis', 0.000263516)
('brags', 0.000263516)
('resonates', 0.00026351595)
('him…', 0.00026351595)
('fighting', 0.00026351577)
('1948', 0.00026351569)
('jousting', 0.00026351566)
('eyed', 0.00026351566)
('collages', 0.0002635156)
('fragmented', 0.00026351

In [207]:

for i in range(2488, 2540):
    print(l0[i])

('ended', 0.00015037404)
('hard—', 0.00015037396)
('hierarchies', 0.00015037395)
('dezvoltării', 0.00015037395)
('creatures', 0.00015037393)
('alvin', 0.00015037393)
('solar', 0.00015037393)
('hymen', 0.0001503739)
('colorfully', 0.0001503739)
('loads', 0.00015037389)
('moods', 0.00015037387)
('route', 0.00015037387)
('bottles—drinks', 0.00015037385)
('poverty—well', 0.0001503738)
('butterflies', 0.0001503738)
('nightingales', 0.0001503738)
('incompetent', 0.00015037374)
('grace', 0.00015037374)
('appealing', 0.00015037374)
('talks', 0.00015037373)
('worry', 0.00015037373)
('brushed', 0.0001503737)
('wondrously', 0.0001503737)
('esperanza', 0.00015037367)
('hunger', 0.00015037366)
('grave', 0.00015037366)
('scan', 0.00015037364)
('acts', 0.00015037364)
('converse', 0.0001503736)
('contented', 0.00015037358)
('alexandrian', 0.00015037357)
('telephoto', 0.00015037355)
('reciter', 0.00015037355)
('students', 0.00015037355)
('wrinkles', 0.00015037353)
('rochester', 0.0001503735)
('horizon'

In [206]:

for i in range(2550, 2592):
    print(l1[i])

('stanza', 0.0002635255)
('done—', 0.0002635255)
('către', 0.0002635254)
('interiorexterior', 0.00026352538)
('horizon', 0.00026352535)
('unfaltering', 0.00026352535)
('clocks', 0.00026352514)
('wrinkles', 0.000263525)
('rochester', 0.0002635249)
('contented', 0.0002635248)
('reciter', 0.0002635248)
('telephoto', 0.00026352477)
('converse', 0.00026352468)
('alexandrian', 0.00026352462)
('students', 0.0002635246)
('grave', 0.00026352436)
('hunger', 0.00026352433)
('scan', 0.0002635243)
('acts', 0.00026352415)
('esperanza', 0.0002635241)
('wondrously', 0.0002635241)
('brushed', 0.00026352392)
('incompetent', 0.0002635239)
('worry', 0.0002635239)
('talks', 0.00026352383)
('appealing', 0.0002635238)
('grace', 0.00026352375)
('poverty—well', 0.00026352354)
('butterflies', 0.0002635235)
('nightingales', 0.0002635233)
('bottles—drinks', 0.00026352316)
('loads', 0.00026352316)
('moods', 0.00026352308)
('colorfully', 0.00026352302)
('route', 0.00026352302)
('hymen', 0.00026352287)
('solar', 0.0

In [208]:

for i in range(3785):
    if l0[i][0] == "fourbeat":
        print(i)

2546


In [209]:

for i in range(3785):
    if l1[i][0] == "fourbeat":
        print(i)

2535


In [210]:

for i in range(2525, 2566):
    print(l0[i])

('unfaltering', 0.00015037347)
('clocks', 0.00015037347)
('done—', 0.00015037345)
('interiorexterior', 0.00015037344)
('stanza', 0.00015037342)
('către', 0.00015037341)
('fumbling', 0.0001503734)
('amount', 0.0001503734)
('drank', 0.0001503734)
('crying', 0.0001503734)
('ferry', 0.00015037334)
('carefully', 0.00015037332)
('nobility', 0.0001503733)
('lamp', 0.00015037326)
('astral', 0.00015037326)
('either—', 0.00015037325)
('riposte', 0.00015037318)
('discouraging', 0.00015037318)
('besiege', 0.00015037315)
('darkyears', 0.00015037313)
('dirtied', 0.00015037313)
('fourbeat', 0.00015037313)
('served', 0.00015037313)
('lens', 0.00015037312)
('undiscovered', 0.0001503731)
('outline', 0.00015037309)
('flutter', 0.00015037306)
('series', 0.00015037306)
('jockey', 0.00015037306)
('waste', 0.00015037305)
('deafening', 0.00015037305)
('steve', 0.00015037303)
('visible', 0.00015037302)
('convulsive', 0.00015037302)
('poems—it', 0.00015037299)
('meagre', 0.00015037299)
('maramureș', 0.000150372

In [211]:

for i in range(2514, 2555):
    print(l1[i])

('looms', 0.00026352832)
('genitalia', 0.00026352826)
('maramureș', 0.0002635281)
('projecting', 0.0002635281)
('allpowerful', 0.00026352803)
('festering', 0.00026352797)
('poems—it', 0.00026352788)
('meagre', 0.00026352782)
('steve', 0.00026352765)
('visible', 0.0002635276)
('convulsive', 0.00026352747)
('waste', 0.00026352744)
('series', 0.00026352744)
('deafening', 0.0002635274)
('flutter', 0.00026352738)
('jockey', 0.00026352738)
('served', 0.0002635272)
('outline', 0.00026352712)
('undiscovered', 0.00026352704)
('lens', 0.000263527)
('dirtied', 0.00026352695)
('fourbeat', 0.00026352686)
('riposte', 0.00026352683)
('darkyears', 0.00026352683)
('discouraging', 0.00026352674)
('besiege', 0.00026352666)
('either—', 0.00026352637)
('astral', 0.00026352634)
('lamp', 0.00026352625)
('nobility', 0.0002635262)
('carefully', 0.00026352602)
('ferry', 0.00026352596)
('crying', 0.00026352578)
('drank', 0.00026352567)
('amount', 0.00026352555)
('fumbling', 0.00026352552)
('stanza', 0.0002635255

In [215]:

for i in range(3785):
    if l0[i][0] == "states":
        print(i)

3345


In [216]:

for i in range(3785):
    if l1[i][0] == "states":
        print(i)

1734


In [217]:

for i in range(3324, 3366):
    print(l0[i])

('offered', 0.00015035881)
('menacing', 0.00015035877)
('rambled', 0.0001503587)
('radically', 0.00015035868)
('apartment', 0.00015035868)
('courbet', 0.00015035864)
('epsom', 0.00015035858)
('stifling', 0.00015035857)
('xxxvi', 0.00015035852)
('boundless', 0.00015035852)
('spats', 0.0001503585)
('rotten', 0.00015035848)
('structure—all', 0.00015035845)
('cheek', 0.00015035842)
('rewriting', 0.0001503584)
('stinking', 0.0001503584)
('early', 0.0001503584)
('rubble', 0.00015035836)
('argued', 0.00015035835)
('realism', 0.00015035833)
('hanksa', 0.00015035833)
('states', 0.00015035828)
('legroom', 0.00015035826)
('rebirth—', 0.00015035823)
('reinforced', 0.00015035819)
('joys', 0.00015035816)
('propeller', 0.00015035813)
('plain', 0.00015035809)
('slouchy', 0.00015035806)
('textual', 0.000150358)
('victims', 0.00015035797)
('rarely', 0.00015035796)
('wave', 0.00015035794)
('raise', 0.00015035793)
('fans', 0.0001503579)
('offend', 0.00015035785)
('portion', 0.00015035782)
('paired', 0.000

In [218]:

for i in range(1713, 1755):
    print(l1[i])

('knife', 0.00026360695)
('divine', 0.0002636069)
('tickling', 0.00026360666)
('pockets', 0.0002636066)
('paired', 0.00026360658)
('amiss', 0.00026360646)
('portion', 0.00026360634)
('offend', 0.00026360605)
('wave', 0.0002636058)
('fans', 0.0002636058)
('rarely', 0.00026360576)
('raise', 0.00026360573)
('victims', 0.0002636057)
('textual', 0.00026360544)
('slouchy', 0.00026360518)
('plain', 0.0002636049)
('propeller', 0.0002636047)
('joys', 0.0002636046)
('reinforced', 0.0002636045)
('rebirth—', 0.00026360428)
('legroom', 0.00026360413)
('states', 0.000263604)
('hanksa', 0.0002636038)
('realism', 0.00026360378)
('argued', 0.00026360372)
('rubble', 0.00026360364)
('early', 0.0002636034)
('stinking', 0.00026360335)
('rewriting', 0.00026360332)
('cheek', 0.00026360326)
('structure—all', 0.00026360308)
('rotten', 0.00026360305)
('xxxvi', 0.00026360282)
('spats', 0.00026360276)
('boundless', 0.00026360276)
('stifling', 0.00026360265)
('epsom', 0.00026360244)
('courbet', 0.00026360215)
('ap

In [219]:

for i in range(3785):
    if l0[i][0] == "verse":
        print(i)

0


In [220]:
for i in range(3785):
    if l1[i][0] == "verse":
        print(i)

0


In [221]:

for i in range(42):
    print(l0[i])

('verse', 0.005966314)
('poet', 0.005565974)
('ovid', 0.0049642525)
('when', 0.004664056)
('other', 0.004160677)
('time', 0.0039610923)
('poem', 0.0035603365)
('work', 0.003258356)
('know', 0.0029588894)
('write', 0.0026574135)
('poets', 0.0025569715)
('naum', 0.0025567992)
('said', 0.002456694)
('poetry', 0.002356873)
('long', 0.0022569722)
('where', 0.0022569187)
('still', 0.0022564116)
('better', 0.0022562025)
('lines', 0.00215656)
('marlowe', 0.0021565487)
('writing', 0.0021560013)
('down', 0.0020562706)
('night', 0.0020560983)
('half', 0.0020560639)
('full', 0.002055618)
('over', 0.0019557113)
('every', 0.0019556393)
('both', 0.0019554493)
('thought', 0.0019554475)
('under', 0.0019552922)
('dryden', 0.0019552046)
('life', 0.0018550719)
('line', 0.001855044)
('words', 0.0018550262)
('between', 0.0018550013)
('sequence', 0.0017553684)
('things', 0.0017553205)
('might', 0.0017553084)
('make', 0.0017551153)
('right', 0.0016551316)
('before', 0.0016550309)
('here', 0.0016548585)


In [222]:

for i in range(42):
    print(l1[i])

('verse', 0.00030230705)
('poet', 0.00029564928)
('other', 0.0002938704)
('ovid', 0.00029200802)
('time', 0.0002874832)
('work', 0.00028705088)
('when', 0.0002867225)
('poem', 0.00028300693)
('naum', 0.00028044012)
('poets', 0.00027954704)
('write', 0.00027933918)
('said', 0.0002788923)
('know', 0.00027794047)
('better', 0.0002772503)
('writing', 0.0002761919)
('still', 0.00027615283)
('dryden', 0.00027614806)
('full', 0.00027609136)
('poetry', 0.0002758482)
('under', 0.00027568787)
('between', 0.0002751058)
('love', 0.00027499296)
('words', 0.00027497308)
('line', 0.00027487817)
('thought', 0.00027487686)
('both', 0.00027487017)
('life', 0.00027473958)
('every', 0.00027387086)
('half', 0.0002737533)
('night', 0.00027358005)
('where', 0.00027349978)
('over', 0.0002734988)
('marlowe', 0.00027333092)
('never', 0.0002733227)
('lines', 0.00027327114)
('long', 0.00027321812)
('only', 0.0002731033)
('water', 0.00027283418)
('down', 0.00027267676)
('each', 0.00027265097)
('something', 0.00027

In [26]:

for i in range(3785):
    if l0[i][0] == "repetition":
        print(i)

720


In [28]:

for i in range(3785):
    if l1[i][0] == "repetition":
        print(i)

1275


In [30]:

for i in range(699, 741):
    print(l0[i])

('american', 0.0002507637)
('incredible', 0.00025074606)
('wheels', 0.00025074455)
('bearer', 0.0002507409)
('held', 0.00025074056)
('favourite', 0.0002507385)
('rifle', 0.00025073625)
('deeply', 0.00025073488)
('food', 0.00025073462)
('fast', 0.00025073337)
('blanket', 0.00025073238)
('migrations', 0.0002507312)
('close', 0.00025073107)
('edited', 0.0002507301)
('wearing', 0.0002507249)
('tunes', 0.00025072432)
('artistic', 0.0002507243)
('whoever', 0.00025072362)
('inspiring', 0.00025072202)
('terms', 0.00025072176)
('whereas', 0.0002507208)
('repetition', 0.0002507172)
('staid', 0.0002507151)
('fruit', 0.00025071492)
('approached', 0.00025071454)
('notes', 0.00025071396)
('rough', 0.00025071367)
('couple', 0.00025071355)
('grandfather', 0.00025071332)
('during', 0.00025071326)
('pool', 0.00025071218)
('velvet', 0.00025071175)
('bearing', 0.00025071163)
('insectpoet', 0.00025071134)
('gives', 0.00025071017)
('covent', 0.00025070948)
('birds', 0.00025070825)
('conducive', 0.0002507071

In [29]:

for i in range(1254, 1296):
    print(l1[i])

('ashy', 0.00026387544)
('multicultural', 0.0002638733)
('longer', 0.0002638723)
('lock', 0.00026387197)
('conducive', 0.0002638707)
('birds', 0.000263865)
('covent', 0.00026385873)
('gives', 0.00026385512)
('insectpoet', 0.00026384916)
('bearing', 0.00026384738)
('velvet', 0.00026384695)
('pool', 0.00026384444)
('during', 0.0002638391)
('grandfather', 0.00026383883)
('couple', 0.00026383688)
('rough', 0.0002638368)
('notes', 0.0002638356)
('approached', 0.00026383225)
('fruit', 0.00026383038)
('staid', 0.00026382942)
('spread', 0.00026382573)
('repetition', 0.00026381848)
('bedded', 0.00026380943)
('heed', 0.00026380707)
('whereas', 0.00026379953)
('instant', 0.0002637981)
('terms', 0.0002637947)
('inspiring', 0.00026379334)
('events', 0.00026378935)
('whoever', 0.0002637849)
('goal—', 0.00026378268)
('artistic', 0.00026378097)
('readership', 0.00026378082)
('tunes', 0.00026378073)
('wearing', 0.0002637784)
('assured', 0.00026377087)
('perpetual', 0.000263765)
('lively', 0.00026375666

In [32]:

for i in range(3785):
    if l0[i][0] == "fumblings":
        print(i)

2061


In [33]:

for i in range(3785):
    if l1[i][0] == "fumblings":
        print(i)

3021


In [34]:

for i in range(2040, 2082):
    print(l0[i])

('laps', 0.00015038122)
('literatures', 0.00015038122)
('selfeffacingly', 0.0001503812)
('cliffs', 0.0001503812)
('turbulence', 0.0001503812)
('makes', 0.0001503812)
('figures', 0.00015038117)
('smoked', 0.00015038115)
('attracts', 0.00015038114)
('objects', 0.0001503811)
('heaps', 0.0001503811)
('moves', 0.00015038108)
('onset', 0.00015038103)
('mending', 0.00015038102)
('halves', 0.000150381)
('bars', 0.000150381)
('introduced', 0.00015038095)
('watermelons', 0.00015038095)
('piecing', 0.00015038095)
('kisses', 0.0001503809)
('ethereal', 0.00015038089)
('fumblings', 0.00015038089)
('strange', 0.00015038083)
('patterns', 0.00015038083)
('dance', 0.00015038083)
('bites', 0.0001503808)
('nobel', 0.00015038079)
('finished', 0.00015038076)
('probing', 0.00015038074)
('survivor', 0.00015038071)
('interview', 0.00015038069)
('insidious', 0.00015038069)
('forcefully', 0.00015038067)
('shifting', 0.00015038064)
('track', 0.00015038061)
('somewhat', 0.00015038058)
('psychiclike', 0.00015038058

In [35]:

for i in range(3000, 3042):
    print(l1[i])

('glory', 0.0002634886)
('stake', 0.0002634886)
('pleasures', 0.00026348856)
('lessons', 0.0002634885)
('somewhat', 0.00026348847)
('psychiclike', 0.00026348833)
('track', 0.00026348824)
('shifting', 0.00026348812)
('forcefully', 0.00026348786)
('insidious', 0.00026348783)
('interview', 0.00026348775)
('survivor', 0.00026348772)
('probing', 0.0002634875)
('finished', 0.00026348748)
('nobel', 0.00026348722)
('strange', 0.0002634872)
('bites', 0.00026348716)
('dance', 0.00026348708)
('patterns', 0.00026348693)
('ethereal', 0.0002634868)
('kisses', 0.00026348679)
('fumblings', 0.00026348673)
('watermelons', 0.00026348646)
('piecing', 0.00026348638)
('introduced', 0.0002634862)
('mending', 0.00026348612)
('halves', 0.00026348612)
('bars', 0.0002634861)
('onset', 0.00026348597)
('moves', 0.00026348562)
('objects', 0.0002634856)
('attracts', 0.0002634856)
('heaps', 0.00026348553)
('figures', 0.00026348533)
('smoked', 0.00026348527)
('makes', 0.00026348516)
('selfeffacingly', 0.00026348513)
(

In [37]:

for i in range(3785):
    if l0[i][0] == "bites":
        print(i)

2065


In [36]:

for i in range(3785):
    if l1[i][0] == "bites":
        print(i)

3016


In [40]:

for i in range(2044, 2086):
    print(l0[i])

('turbulence', 0.0001503812)
('makes', 0.0001503812)
('figures', 0.00015038117)
('smoked', 0.00015038115)
('attracts', 0.00015038114)
('objects', 0.0001503811)
('heaps', 0.0001503811)
('moves', 0.00015038108)
('onset', 0.00015038103)
('mending', 0.00015038102)
('halves', 0.000150381)
('bars', 0.000150381)
('introduced', 0.00015038095)
('watermelons', 0.00015038095)
('piecing', 0.00015038095)
('kisses', 0.0001503809)
('ethereal', 0.00015038089)
('fumblings', 0.00015038089)
('strange', 0.00015038083)
('patterns', 0.00015038083)
('dance', 0.00015038083)
('bites', 0.0001503808)
('nobel', 0.00015038079)
('finished', 0.00015038076)
('probing', 0.00015038074)
('survivor', 0.00015038071)
('interview', 0.00015038069)
('insidious', 0.00015038069)
('forcefully', 0.00015038067)
('shifting', 0.00015038064)
('track', 0.00015038061)
('somewhat', 0.00015038058)
('psychiclike', 0.00015038058)
('pleasures', 0.00015038057)
('lessons', 0.00015038054)
('glory', 0.00015038051)
('stake', 0.00015038051)
('tho

In [39]:

for i in range(2980, 3060):
    print(l1[i])

('takeover', 0.0002634905)
('aches', 0.00026349048)
('abhorrent', 0.00026349048)
('drummers', 0.00026348996)
('palate', 0.0002634898)
('nearby', 0.0002634898)
('1971', 0.00026348972)
('shorter', 0.00026348972)
('eclipses', 0.0002634897)
('record', 0.0002634897)
('healthier', 0.0002634897)
('develops', 0.00026348964)
('intuition', 0.0002634896)
('planned', 0.00026348958)
('posts', 0.00026348946)
('naked', 0.00026348935)
('length', 0.000263489)
('thomas', 0.00026348894)
('vicarious', 0.00026348885)
('newspaper', 0.00026348876)
('glory', 0.0002634886)
('stake', 0.0002634886)
('pleasures', 0.00026348856)
('lessons', 0.0002634885)
('somewhat', 0.00026348847)
('psychiclike', 0.00026348833)
('track', 0.00026348824)
('shifting', 0.00026348812)
('forcefully', 0.00026348786)
('insidious', 0.00026348783)
('interview', 0.00026348775)
('survivor', 0.00026348772)
('probing', 0.0002634875)
('finished', 0.00026348748)
('nobel', 0.00026348722)
('strange', 0.0002634872)
('bites', 0.00026348716)
('dance'

In [47]:

for i in range(3785):
    if l0[i][0] == "cast":
        print(i)

3761


In [48]:

for i in range(3785):
    if l1[i][0] == "cast":
        print(i)

1309


In [49]:

for i in range(3739, 3786):
    print(l0[i])

('library', 0.00015033888)
('awkward', 0.00015033863)
('zeppelin', 0.0001503386)
('alarmed', 0.00015033853)
('clichédismantling', 0.00015033822)
('chat', 0.00015033808)
('jasmine', 0.0001503379)
('4year', 0.00015033786)
('blade', 0.00015033744)
('cages', 0.00015033729)
('deserves', 0.00015033729)
('rustling', 0.00015033715)
('degas', 0.00015033701)
('conservation', 0.00015033691)
('editing', 0.00015033691)
('foremost', 0.00015033678)
('schimbarea', 0.0001503365)
('smelling', 0.00015033648)
('alchemist', 0.00015033611)
('ceremonies', 0.00015033563)
('informing', 0.00015033524)
('contain', 0.00015033461)
('cast', 0.00015033425)
('flooded', 0.00015033422)
('tones', 0.00015033397)
('bounty', 0.00015033394)
('flares', 0.00015033355)
('heine', 0.00015033336)
('songs', 0.00015033329)
('bone', 0.00015033325)
('king', 0.0001503328)
('linen', 0.00015033186)
('ascet', 0.0001503314)
('realm', 0.00015033118)
('dyed', 0.00015033057)
('translatingwriting', 0.0001503301)
('lively', 0.00015032884)
('pe

IndexError: list index out of range

In [50]:

for i in range(1288, 1330):
    print(l1[i])

('wearing', 0.0002637784)
('assured', 0.00026377087)
('perpetual', 0.000263765)
('lively', 0.00026375666)
('edited', 0.0002637507)
('translatingwriting', 0.0002637501)
('dyed', 0.00026374782)
('close', 0.00026374563)
('migrations', 0.00026374546)
('realm', 0.00026374465)
('ascet', 0.00026374342)
('linen', 0.00026374112)
('blanket', 0.00026373914)
('king', 0.0002637362)
('bone', 0.00026373402)
('fast', 0.00026373393)
('songs', 0.00026373347)
('heine', 0.0002637333)
('flares', 0.00026373222)
('bounty', 0.00026373018)
('tones', 0.00026372992)
('cast', 0.00026372873)
('flooded', 0.0002637287)
('food', 0.0002637273)
('contain', 0.0002637268)
('deeply', 0.00026372581)
('informing', 0.00026372354)
('ceremonies', 0.00026372142)
('rifle', 0.00026371935)
('alchemist', 0.000263719)
('schimbarea', 0.00026371705)
('smelling', 0.000263717)
('foremost', 0.00026371563)
('conservation', 0.00026371493)
('editing', 0.00026371493)
('degas', 0.00026371438)
('rustling', 0.00026371362)
('deserves', 0.0002637

In [64]:

for i in range(3785):
    if l0[i][0] == "honour":
        print(i)

1379


In [63]:

for i in range(3785):
    if l1[i][0] == "honour":
        print(i)

3701


In [190]:

for i in range(3675, 3785):
    print(l0[i])

('abramovic', 0.00015034521)
('herself', 0.0001503452)
('edits', 0.00015034515)
('eliza', 0.00015034508)
('gradation', 0.00015034508)
('rash', 0.00015034497)
('purple', 0.00015034486)
('seats', 0.00015034481)
('view—from', 0.0001503447)
('wringing', 0.0001503445)
('town', 0.00015034447)
('hendrix', 0.00015034447)
('acoustic', 0.00015034442)
('belly', 0.00015034433)
('logistical', 0.00015034419)
('laocoön', 0.00015034419)
('spray', 0.00015034417)
('opinion—mere', 0.00015034415)
('selfimposed', 0.00015034412)
('tourists', 0.00015034404)
('searching', 0.00015034393)
('meal', 0.00015034381)
('happening', 0.00015034371)
('scarcely', 0.00015034368)
('noon', 0.00015034368)
('multiplication', 0.00015034364)
('newly', 0.00015034355)
('nocturnal', 0.00015034349)
('powders', 0.00015034343)
('transbiographical', 0.00015034332)
('amounted', 0.00015034324)
('birthmark', 0.00015034324)
('mesh', 0.00015034318)
('perform', 0.00015034314)
('runway', 0.00015034311)
('worldold', 0.00015034301)
('leafy', 0

In [191]:

for i in range(3574, 3674):
    print(l0[i])

('able', 0.00015035084)
('duty', 0.00015035081)
('mincemeat', 0.00015035077)
('eddie', 0.00015035075)
('bizarre', 0.00015035074)
('1915', 0.00015035072)
('frustrations', 0.0001503507)
('monologues', 0.00015035068)
('shot', 0.00015035065)
('heartaches', 0.0001503506)
('wight', 0.00015035058)
('personally', 0.00015035058)
('it—say', 0.00015035053)
('harm', 0.0001503505)
('doctoral', 0.00015035043)
('recount', 0.0001503504)
('beard', 0.00015035037)
('horus', 0.00015035037)
('grammar', 0.00015035036)
('merciless', 0.00015035033)
('motorcycle', 0.00015035033)
('occupation', 0.00015035029)
('restless', 0.00015035027)
('witless', 0.00015035021)
('murder', 0.00015035013)
('opens', 0.00015035)
('certainly', 0.00015035)
('amplified', 0.00015034991)
('jokes', 0.00015034984)
('throats', 0.00015034976)
('drier', 0.00015034972)
('turner', 0.00015034966)
('explores', 0.00015034966)
('hated', 0.00015034959)
('vocalist', 0.00015034954)
('john', 0.00015034944)
('marlowedryden', 0.00015034943)
('shovel',

In [27]:

for i in range(3675, 3785):
    print(l1[i])

('drums', 0.00026339912)
('wanting—', 0.00026339907)
('saved', 0.00026339837)
('stretching', 0.00026339805)
('immersion', 0.000263398)
('conceptions', 0.00026339793)
('ballet', 0.0002633971)
('motif', 0.00026339706)
('hates', 0.0002633967)
('continue', 0.00026339665)
('lonely', 0.00026339662)
('enjoy', 0.0002633966)
('exposed', 0.00026339645)
('intoxication', 0.00026339592)
('searing', 0.00026339592)
('liverpool', 0.00026339505)
('fluctuating', 0.00026339415)
('applause', 0.0002633935)
('describe', 0.000263393)
('drafted', 0.0002633925)
('archive', 0.00026339188)
('aura', 0.00026339162)
('buttons', 0.00026339135)
('yelping', 0.00026339092)
('fame', 0.00026338943)
('nuance', 0.00026338873)
('honour', 0.00026338833)
('unhappy', 0.00026338824)
('musicians', 0.00026338824)
('claim', 0.0002633874)
('forty', 0.0002633873)
('offspring', 0.00026338655)
('approved', 0.00026338646)
('contents', 0.0002633864)
('urbane', 0.000263386)
('219', 0.0002633858)
('proximate', 0.00026338524)
('barefoot', 